# **Calculator**

In [2]:
# @title Default title text
import math
import fractions
import re
import ipywidgets as widgets
from IPython.display import display, HTML

# --- Adjustable Variables ---
BOLD_WEIGHT = "1000"  # Adjust this value (e.g., "normal", "bold", "700", etc.)
BUTTON_TEXT_STYLE = {'font_weight': BOLD_WEIGHT}
BUTTON_WIDTH = '150px'
BUTTON_HEIGHT = '40px'
small_button_layout = widgets.Layout(width=BUTTON_WIDTH, height=BUTTON_HEIGHT)

# --- Global State ---
calc_state = {
    "expression": "",
    "result": ""
}

# --- Display Widgets ---
display(HTML("""
<style>
    .big-input input {
        font-size: 24px !important;
        height: 60px !important;
        line-height: 70px !important;
        padding: 5px 10px !important;
        text-align: right !important;
    }
</style>
"""))

expression_input = widgets.Text(
    value="",
    placeholder="Enter expression",
    layout=widgets.Layout(width='99.1%', height='60px')
)
expression_input.add_class("big-input")
display_style = ("text-align:right; font-size:32px; color:white; background:#1a1a1a; "
                 "padding:10px; border-radius:8px; min-height:45px;")
result_display = widgets.HTML(value=f"<div style='{display_style}'>{calc_state['result']}</div>")

def update_display():
    expression_input.value = calc_state["expression"]
    result_display.value = f"<div style='{display_style}'>{calc_state['result'] if calc_state['result'] else '&nbsp;'}</div>"

def on_expr_change(change):
    if change['name'] == 'value':
        calc_state["expression"] = change['new']
expression_input.observe(on_expr_change, names='value')

# --- Calculator Editing Functions ---
def append_to_expr(val):
    expr = expression_input.value
    calc_state["expression"] = expr + val
    update_display()

def clear_calc(b=None):
    calc_state["expression"] = ""
    calc_state["result"] = ""
    update_display()

def backspace(b=None):
    calc_state["expression"] = calc_state["expression"][:-1]
    update_display()

# --- Mathematical Helper Functions ---
def fact(x):
    return math.factorial(int(x))

def inv(x):
    return 1 / x

def cbrt(x):
    if x >= 0:
        return x ** (1/3)
    else:
        return -((-x) ** (1/3))

def nth(n, x):
    return x ** (1/n)

def dec2frac(x):
    frac = fractions.Fraction(x).limit_denominator()
    p, q = frac.numerator, frac.denominator
    if abs(p) >= q:
        sign = -1 if p < 0 else 1
        p_abs = abs(p)
        whole = p_abs // q
        remainder = p_abs % q
        if remainder == 0:
            return f"{sign * whole}"
        else:
            if whole != 0:
                return f"{sign * whole} {remainder}/{q} ({p}/{q})"
            else:
                return f"{p}/{q}"
    else:
        return f"{p}/{q}"

# --- Custom Trigonometric Functions (angles in degrees) ---
def sin_deg(x):
    return math.sin(math.radians(x))

def cos_deg(x):
    rad = math.radians(x)
    val = math.cos(rad)
    if abs(val) < 1e-10:
        return 0.0
    return val

def tan_deg(x):
    rad = math.radians(x)
    cos_val = math.cos(rad)
    if abs(cos_val) < 1e-10:
        return float('inf')
    t = math.tan(rad)
    if abs(t - round(t)) < 1e-9:
        t = round(t)
    return t

def asin_deg(x):
    return math.degrees(math.asin(x))

def acos_deg(x):
    return math.degrees(math.acos(x))

def atan_deg(x):
    return math.degrees(math.atan(x))

# --- Preprocess Expression for nCr, nPr, and nth ---
def preprocess_expr(expr):
    expr = re.sub(r'(\d+(?:\.\d+)?)\s*nth\s*(\d+(?:\.\d+)?)', r'nth(\1,\2)', expr)
    expr = re.sub(r'(\d+(?:\.\d+)?)\s*C\s*(\d+(?:\.\d+)?)', r'comb(\1,\2)', expr)
    expr = re.sub(r'(\d+(?:\.\d+)?)\s*P\s*(\d+(?:\.\d+)?)', r'perm(\1,\2)', expr)
    return expr

# --- Allowed Functions for Safe Eval ---
allowed_names = {
    "sin": sin_deg,
    "cos": cos_deg,
    "tan": tan_deg,
    "asin": asin_deg,
    "acos": acos_deg,
    "atan": atan_deg,
    "ln": math.log,
    "log": math.log10,
    "sqrt": math.sqrt,
    "cbrt": cbrt,
    "fact": fact,
    "inv": inv,
    "nth": nth,
    "pi": math.pi,
    "e": math.e,
    "comb": lambda n, r: math.comb(int(n), int(r)),
    "perm": lambda n, r: math.perm(int(n), int(r)) if hasattr(math, "perm")
                           else math.factorial(int(n)) // math.factorial(int(n)-int(r))
}

# --- Calculation ---
def calculate(b=None):
    try:
        expr = preprocess_expr(calc_state["expression"])
        result = eval(expr, {"__builtins__": None}, allowed_names)
        # If result is a float and nearly equal to an integer, round it.
        if isinstance(result, float) and abs(result - round(result)) < 1e-9:
            result = round(result)
        calc_state["result"] = str(result)
        update_display()
    except Exception:
        calc_state["result"] = "Error"
        update_display()

def convert_to_fraction(b=None):
    try:
        value = float(calc_state["result"])
        frac_str = dec2frac(value)
        calc_state["result"] = frac_str
        update_display()
    except Exception:
        calc_state["result"] = "Error"
        update_display()

# --- Unary Operation Helpers for Factorial and Inverse ---
def apply_unary_operation(op):
    """Replace the last number in the expression with op(number)."""
    expr = calc_state["expression"]
    # This regex matches the last number (integer or decimal) at the end of the expression.
    match = re.search(r'(-?\d+(?:\.\d+)?)\s*$', expr)
    if match:
        num_str = match.group(1)
        new_expr = expr[:match.start()] + f"{op}({num_str})"
        calc_state["expression"] = new_expr
        update_display()

def apply_factorial(b=None):
    apply_unary_operation("fact")

def apply_inverse(b=None):
    apply_unary_operation("inv")

# --- Create Buttons with BUTTON_TEXT_STYLE applied ---
btn_clear = widgets.Button(description="C", button_style="warning", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_back = widgets.Button(description="⌫", button_style="warning", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_clear.on_click(clear_calc)
btn_back.on_click(backspace)

btn_asin = widgets.Button(description="sin⁻¹", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_acos = widgets.Button(description="cos⁻¹", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_atan = widgets.Button(description="tan⁻¹", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_asin.on_click(lambda b: append_to_expr("asin("))
btn_acos.on_click(lambda b: append_to_expr("acos("))
btn_atan.on_click(lambda b: append_to_expr("atan("))

btn_sin = widgets.Button(description="sin", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_cos = widgets.Button(description="cos", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_tan = widgets.Button(description="tan", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_ln  = widgets.Button(description="ln", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_log = widgets.Button(description="log", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_sin.on_click(lambda b: append_to_expr("sin("))
btn_cos.on_click(lambda b: append_to_expr("cos("))
btn_tan.on_click(lambda b: append_to_expr("tan("))
btn_ln.on_click(lambda b: append_to_expr("ln("))
btn_log.on_click(lambda b: append_to_expr("log("))

btn_sqrt = widgets.Button(description="√", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_cbrt = widgets.Button(description="∛", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_nth = widgets.Button(description="nth", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_frac = widgets.Button(description="Frac", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_sqrt.on_click(lambda b: append_to_expr("sqrt("))
btn_cbrt.on_click(lambda b: append_to_expr("cbrt("))
btn_nth.on_click(lambda b: append_to_expr(" nth "))
btn_frac.on_click(convert_to_fraction)

btn_x2   = widgets.Button(description="x²", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_x3   = widgets.Button(description="x³", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_xy   = widgets.Button(description="x^y", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_fact = widgets.Button(description="!", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_inv  = widgets.Button(description="1/x", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_x2.on_click(lambda b: append_to_expr("**2"))
btn_x3.on_click(lambda b: append_to_expr("**3"))
btn_xy.on_click(lambda b: append_to_expr("**"))
# Instead of appending text, apply the operation on the last number.
btn_fact.on_click(apply_factorial)
btn_inv.on_click(apply_inverse)

btn_nCr  = widgets.Button(description="nCr", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_nPr  = widgets.Button(description="nPr", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_pi   = widgets.Button(description="π", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_e    = widgets.Button(description="e", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_open = widgets.Button(description="(", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_close= widgets.Button(description=")", button_style="info", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_nCr.on_click(lambda b: append_to_expr(" C "))
btn_nPr.on_click(lambda b: append_to_expr(" P "))
btn_pi.on_click(lambda b: append_to_expr("pi"))
btn_e.on_click(lambda b: append_to_expr("e"))
btn_open.on_click(lambda b: append_to_expr("("))
btn_close.on_click(lambda b: append_to_expr(")"))

numeric_buttons = [
    "7", "8", "9", "/",
    "4", "5", "6", "*",
    "1", "2", "3", "-",
    "0", ".", "+", "="
]
num_buttons = []
for desc in numeric_buttons:
    if desc == "=":
        btn = widgets.Button(description=desc, button_style="success", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
        btn.on_click(calculate)
    else:
        btn = widgets.Button(description=desc, layout=small_button_layout, style=BUTTON_TEXT_STYLE)
        btn.on_click(lambda b, d=desc: append_to_expr(d))
    num_buttons.append(btn)

grid = widgets.GridBox(num_buttons,
                         layout=widgets.Layout(
                             grid_template_columns=f"repeat(4, {BUTTON_WIDTH})",
                             grid_gap="5px"
                         ))
grid_box = widgets.HBox([grid], layout=widgets.Layout(justify_content='flex-end'))

btn_equals_top = widgets.Button(description="=", button_style="success", layout=small_button_layout, style=BUTTON_TEXT_STYLE)
btn_equals_top.on_click(calculate)

top_row = widgets.HBox([btn_equals_top, btn_clear, btn_back], layout=widgets.Layout(justify_content='flex-end'))
inv_trig_row = widgets.HBox([btn_asin, btn_acos, btn_atan], layout=widgets.Layout(justify_content='flex-end'))
func_row1 = widgets.HBox([btn_sin, btn_cos, btn_tan, btn_ln, btn_log], layout=widgets.Layout(justify_content='flex-end'))
func_row2 = widgets.HBox([btn_sqrt, btn_cbrt, btn_nth, btn_frac], layout=widgets.Layout(justify_content='flex-end'))
func_row3 = widgets.HBox([btn_x2, btn_x3, btn_xy, btn_fact, btn_inv], layout=widgets.Layout(justify_content='flex-end'))
func_row4 = widgets.HBox([btn_nCr, btn_nPr, btn_pi, btn_e, btn_open, btn_close], layout=widgets.Layout(justify_content='flex-end'))

# Assemble the UI with the new editable expression input at the top.
calc_ui = widgets.VBox([
    expression_input,
    result_display,
    top_row,
    inv_trig_row,
    func_row1,
    func_row2,
    func_row3,
    func_row4,
    grid_box
])
update_display()
display(calc_ui)

# **STATS**

In [8]:
# @title Default title text
# Install ipywidgets if needed (uncomment the next two lines if running in an environment without ipywidgets)
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

import ipywidgets as widgets
from ipywidgets import VBox, HBox, Tab, Layout
import numpy as np
from scipy.stats import norm, binom, poisson, expon, uniform, geom, hypergeom
import math
from IPython.display import display

####################################
# Distribution Calculator UI Section
####################################

# Distribution selection widget
dist_dropdown = widgets.Dropdown(
    options=[
        ("Normal Distribution", "normal"),
        ("Binomial Distribution", "binomial"),
        ("Poisson Distribution", "poisson"),
        ("Exponential Distribution", "exponential"),
        ("Uniform Distribution", "uniform"),
        ("Discrete Uniform Distribution", "discrete_uniform"),
        ("Geometric Distribution", "geometric"),
        ("Hypergeometric Distribution", "hypergeometric"),
        ("Beta Distribution", "beta")
    ],
    description="Distribution:",
    style={'description_width': 'initial'}
)

# Function type widget: Choose which function to compute
# (New options "PPF" and "ISF" have been added.)
func_dropdown = widgets.Dropdown(
    options=[
        ("CDF", "cdf"),
        ("1 - CDF", "one_minus_cdf"),
        ("PDF/PMF", "pdf"),
        ("PPF", "ppf"),
        ("ISF", "isf")
    ],
    description="Function Type:",
    style={'description_width': 'initial'}
)

# Container for parameter widgets (will be updated based on distribution)
param_box = VBox([])

# For normal distribution extra input mode
# (This widget will only be used when dist == "normal")
normal_mode = widgets.ToggleButtons(
    options=["Raw", "Z-score", "Percentile"],
    description="Normal Input Mode:",
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)
# Container for normal-specific parameter inputs
normal_param_box = VBox([], layout=Layout(margin='10px 0px'))

def update_normal_params(*args):
    """Update the parameter widgets for Normal distribution based on the chosen input mode."""
    mode = normal_mode.value
    if mode == "Raw":
        mean_widget = widgets.FloatText(description="Mean (μ):", value=0.0, layout=Layout(width='300px'))
        std_widget = widgets.FloatText(description="Std Dev (σ):", value=1.0, layout=Layout(width='300px'))
        x_widget = widgets.FloatText(description="x:", value=0.0, layout=Layout(width='300px'))
        normal_param_box.children = [mean_widget, std_widget, x_widget]
    elif mode == "Z-score":
        z_widget = widgets.FloatText(description="z-score:", value=0.0, layout=Layout(width='300px'))
        normal_param_box.children = [z_widget]
    elif mode == "Percentile":
        perc_widget = widgets.BoundedFloatText(description="Percentile:", min=0.0, max=1.0, step=0.01, value=0.5, layout=Layout(width='300px'))
        normal_param_box.children = [perc_widget]

# Update normal-specific parameters when mode changes
normal_mode.observe(update_normal_params, names="value")
update_normal_params()  # initialize with default ("Raw")

def update_params(*args):
    """Update parameter widgets based on the chosen distribution."""
    dist = dist_dropdown.value
    widgets_list = []

    if dist == "normal":
        # For normal distribution, include the extra input mode toggle and container
        widgets_list = [normal_mode, normal_param_box]
    elif dist == "binomial":
        n_widget = widgets.IntText(description="n (trials):", value=10, layout=Layout(width='300px'))
        p_widget = widgets.BoundedFloatText(description="p (prob):", min=0.0, max=1.0, step=0.01, value=0.5, layout=Layout(width='300px'))
        x_widget = widgets.IntText(description="x (successes):", value=5, layout=Layout(width='300px'), style={'description_width': '150px'})
        widgets_list = [n_widget, p_widget, x_widget]
    elif dist == "poisson":
        lam_widget = widgets.FloatText(description="λ (rate):", value=3.0, layout=Layout(width='300px'))
        x_widget = widgets.IntText(description="x:", value=2, layout=Layout(width='300px'))
        widgets_list = [lam_widget, x_widget]
    elif dist == "exponential":
        lam_widget = widgets.FloatText(description="λ (rate):", value=1.0, layout=Layout(width='300px'))
        x_widget = widgets.FloatText(description="x:", value=0.0, layout=Layout(width='300px'))
        widgets_list = [lam_widget, x_widget]
    elif dist == "uniform":
        lower_widget = widgets.FloatText(description="Lower bound (a):", value=0.0, layout=Layout(width='300px'), style={'description_width': '150px'})
        upper_widget = widgets.FloatText(description="Upper bound (b):", value=1.0, layout=Layout(width='300px'), style={'description_width': '150px'})
        x_widget = widgets.FloatText(description="x:", value=0.5, layout=Layout(width='300px'))
        widgets_list = [lower_widget, upper_widget, x_widget]
    elif dist == "discrete_uniform":
        lower_widget = widgets.IntText(description="Lower bound (a):", value=1, layout=Layout(width='300px'), style={'description_width': '150px'})
        upper_widget = widgets.IntText(description="Upper bound (b):", value=4, layout=Layout(width='300px'), style={'description_width': '150px'})
        x_widget = widgets.IntText(description="x:", value=2, layout=Layout(width='300px'))
        widgets_list = [lower_widget, upper_widget, x_widget]
    elif dist == "geometric":
        p_widget = widgets.BoundedFloatText(description="p (prob):", min=0.0, max=1.0, step=0.01, value=0.5, layout=Layout(width='300px'))
        x_widget = widgets.IntText(description="x (trial number):", value=1, layout=Layout(width='300px'), style={'description_width': '150px'})
        widgets_list = [p_widget, x_widget]
    elif dist == "hypergeometric":
        N_widget = widgets.IntText(description="Population size (N):", value=50, layout=Layout(width='300px'), style={'description_width': '150px'})
        K_widget = widgets.IntText(description="Successes in population (K):", value=10, layout=Layout(width='300px'), style={'description_width': '200px'})
        n_widget = widgets.IntText(description="Sample size (n):", value=5, layout=Layout(width='300px'), style={'description_width': '150px'})
        x_widget = widgets.IntText(description="x (successes in sample):", value=2, layout=Layout(width='300px'), style={'description_width': '150px'})
        widgets_list = [N_widget, K_widget, n_widget, x_widget]
    elif dist == "beta":
        a_widget = widgets.FloatText(description="α (shape1):", value=2.0, layout=Layout(width='300px'), style={'description_width': '150px'})
        b_widget = widgets.FloatText(description="β (shape2):", value=5.0, layout=Layout(width='300px'), style={'description_width': '150px'})
        x_widget = widgets.FloatText(description="x:", value=0.5, layout=Layout(width='300px'), style={'description_width': '150px'})
        widgets_list = [a_widget, b_widget, x_widget]

    param_box.children = widgets_list

# Update parameters initially and when distribution selection changes
update_params()
dist_dropdown.observe(update_params, names="value")

# Output widget for displaying the result of the distribution calculation
dist_output = widgets.Output()

def calc_distribution(b):
    dist_output.clear_output()
    with dist_output:
        try:
            dist = dist_dropdown.value
            func_type = func_dropdown.value
            result = None

            if dist == "normal":
                # For normal distribution, check the input mode
                mode = normal_mode.value
                if mode == "Raw":
                    # Expecting [Mean, Std Dev, x]
                    mean = normal_param_box.children[0].value
                    std = normal_param_box.children[1].value
                    x_val = normal_param_box.children[2].value
                    d = norm(loc=mean, scale=std)
                    if func_type == "cdf":
                        result = d.cdf(x_val)
                    elif func_type == "one_minus_cdf":
                        result = 1 - d.cdf(x_val)
                    elif func_type == "pdf":
                        result = d.pdf(x_val)
                    elif func_type == "ppf":
                        result = d.ppf(x_val)
                    elif func_type == "isf":
                        result = d.isf(x_val)
                    print(f"Result: {result}")
                elif mode == "Z-score":
                    # Expecting [z-score]
                    z_val = normal_param_box.children[0].value
                    d = norm(0, 1)
                    if func_type == "cdf":
                        result = d.cdf(z_val)
                    elif func_type == "one_minus_cdf":
                        result = 1 - d.cdf(z_val)
                    elif func_type == "pdf":
                        result = d.pdf(z_val)
                    elif func_type == "ppf":
                        result = d.ppf(z_val)
                    elif func_type == "isf":
                        result = d.isf(z_val)
                    print(f"Result (Standard Normal at input={z_val}): {result}")
                elif mode == "Percentile":
                    # Expecting [Percentile] to convert to z-score using ppf (this mode always computes the inverse CDF)
                    perc = normal_param_box.children[0].value
                    z_val = norm.ppf(perc)
                    print(f"Percentile {perc} corresponds to z-score: {z_val}")
            elif dist == "binomial":
                n = param_box.children[0].value
                p = param_box.children[1].value
                x = param_box.children[2].value
                d = binom(n, p)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                elif func_type == "pdf":
                    result = d.pmf(x)
                elif func_type == "ppf":
                    result = d.ppf(x)
                elif func_type == "isf":
                    result = d.isf(x)
                print(f"Result: {result}")
            elif dist == "poisson":
                lam = param_box.children[0].value
                x = param_box.children[1].value
                d = poisson(lam)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                elif func_type == "pdf":
                    result = d.pmf(x)
                elif func_type == "ppf":
                    result = d.ppf(x)
                elif func_type == "isf":
                    result = d.isf(x)
                print(f"Result: {result}")
            elif dist == "exponential":
                lam = param_box.children[0].value
                x = param_box.children[1].value
                d = expon(scale=1/lam)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                elif func_type == "pdf":
                    result = d.pdf(x)
                elif func_type == "ppf":
                    result = d.ppf(x)
                elif func_type == "isf":
                    result = d.isf(x)
                print(f"Result: {result}")
            elif dist == "uniform":
                a = param_box.children[0].value
                b_val = param_box.children[1].value
                if b_val <= a:
                    raise ValueError("Upper bound must be greater than lower bound.")
                x = param_box.children[2].value
                d = uniform(loc=a, scale=b_val - a)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                elif func_type == "pdf":
                    result = d.pdf(x)
                elif func_type == "ppf":
                    result = d.ppf(x)
                elif func_type == "isf":
                    result = d.isf(x)
                print(f"Result: {result}")
            elif dist == "discrete_uniform":
                a = param_box.children[0].value
                b_val = param_box.children[1].value
                if b_val < a:
                    raise ValueError("Upper bound must be greater than or equal to lower bound.")
                x = param_box.children[2].value
                from scipy.stats import randint
                d = randint(low=a, high=b_val + 1)  # randint upper bound is exclusive
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                elif func_type == "pdf":
                    result = d.pmf(x)
                elif func_type == "ppf":
                    result = d.ppf(x)
                elif func_type == "isf":
                    result = d.isf(x)
                print(f"Result: {result}")
            elif dist == "geometric":
                p = param_box.children[0].value
                x = param_box.children[1].value
                d = geom(p)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                elif func_type == "pdf":
                    result = d.pmf(x)
                elif func_type == "ppf":
                    result = d.ppf(x)
                elif func_type == "isf":
                    result = d.isf(x)
                print(f"Result: {result}")
            elif dist == "hypergeometric":
                N = param_box.children[0].value
                K = param_box.children[1].value
                n = param_box.children[2].value
                x = param_box.children[3].value
                d = hypergeom(N, K, n)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                elif func_type == "pdf":
                    result = d.pmf(x)
                elif func_type == "ppf":
                    result = d.ppf(x)
                elif func_type == "isf":
                    result = d.isf(x)
                print(f"Result: {result}")
            elif dist == "beta":
                a_val = param_box.children[0].value
                b_val = param_box.children[1].value
                x = param_box.children[2].value
                from scipy.stats import beta
                d = beta(a_val, b_val)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                elif func_type == "pdf":
                    result = d.pdf(x)
                elif func_type == "ppf":
                    result = d.ppf(x)
                elif func_type == "isf":
                    result = d.isf(x)
                print(f"Result: {result}")
        except Exception as e:
            print(f"Error: {e}")

# Button to trigger distribution calculation
calc_button = widgets.Button(description="Calculate", button_style="success", layout=Layout(width='150px'))
calc_button.on_click(calc_distribution)

# Assemble Distribution Calculator UI
dist_calculator_ui = VBox([
    dist_dropdown,
    func_dropdown,
    param_box,
    calc_button,
    dist_output
], layout=Layout(margin='10px 0px'))

####################################
# Conversion Calculator UI Section
####################################

# Conversion category: either convert a single value or convert parameters.
conv_category = widgets.RadioButtons(
    options=["Value Conversion", "Parameter Conversion"],
    description="Conversion Category:",
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)

# Container to hold the inner conversion UI (which changes based on category)
conv_inner_box = VBox([], layout=Layout(margin='10px 0px'))

# --- For Value Conversion ---
conv_radio_value = widgets.RadioButtons(
    options=[("Normal to Lognormal", "normal_to_lognormal"), ("Lognormal to Normal", "lognormal_to_normal")],
    description="Value Conversion:",
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)
conv_value_input = widgets.FloatText(
    description="Value:",
    value=0.0,
    layout=Layout(width='300px')
)
value_conv_ui = VBox([conv_radio_value, conv_value_input], layout=Layout(margin='10px 0px'))

# --- For Parameter Conversion ---
parameter_conv_radio = widgets.RadioButtons(
    options=[("Lognormal to Normal Parameters", "normal_to_lognormal_params"),
             ("Normal to Lognormal Parameters", "lognormal_to_normal_params")],
    description="Parameter Conversion:",
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)
# Containers for the parameter inputs with fixed width and margin
normal_param_inputs = VBox([
    widgets.FloatText(description="Lognormal Mean (m):", value=0.0, layout=Layout(width='300px'), style={'description_width': '150px'}),
    widgets.FloatText(description="Lognormal Std Dev:", value=1.0, layout=Layout(width='300px'), style={'description_width': '150px'})
], layout=Layout(margin='10px 0px'))
lognormal_param_inputs = VBox([
    widgets.FloatText(description="Normal Mean (μ):", value=1.0, layout=Layout(width='300px'), style={'description_width': '150px'}),
    widgets.FloatText(description="Normal Variance:", value=0.5, layout=Layout(width='300px'), style={'description_width': '150px'})
], layout=Layout(margin='10px 0px'))
# Container that will hold one of the above based on the radio selection.
param_conv_input_box = VBox([], layout=Layout(margin='10px 0px'))

def update_param_conv_inputs(*args):
    """Update parameter conversion inputs based on chosen conversion type."""
    if parameter_conv_radio.value == "normal_to_lognormal_params":
        param_conv_input_box.children = [normal_param_inputs]
    else:
        param_conv_input_box.children = [lognormal_param_inputs]

parameter_conv_radio.observe(update_param_conv_inputs, names="value")
update_param_conv_inputs()

parameter_conv_ui = VBox([parameter_conv_radio, param_conv_input_box], layout=Layout(margin='10px 0px'))

def update_conv_inner(*args):
    """Update the inner conversion UI based on conversion category."""
    if conv_category.value == "Value Conversion":
        conv_inner_box.children = [value_conv_ui]
    else:
        conv_inner_box.children = [parameter_conv_ui]

conv_category.observe(update_conv_inner, names="value")
update_conv_inner()

# Output widget for conversion results
conv_output = widgets.Output()

# Button for performing conversion
conv_button = widgets.Button(description="Convert", button_style="info", layout=Layout(width='150px'))

def perform_conversion(b):
    conv_output.clear_output()
    with conv_output:
        try:
            if conv_category.value == "Value Conversion":
                val = conv_value_input.value
                if conv_radio_value.value == "normal_to_lognormal":
                    # Single value conversion: normal to lognormal using exp.
                    result = math.exp(val)
                    print(f"exp({val}) = {result}")
                else:
                    # Lognormal to normal using log.
                    if val <= 0:
                        raise ValueError("Value must be positive for log conversion.")
                    result = math.log(val)
                    print(f"log({val}) = {result}")
            else:
                # Parameter Conversion
                if parameter_conv_radio.value == "normal_to_lognormal_params":
                    mu = normal_param_inputs.children[0].value
                    sigma = normal_param_inputs.children[1].value
                    lognorm_mean = math.exp(mu + sigma**2 / 2)
                    lognorm_variance = (math.exp(sigma**2) - 1) * math.exp(2*mu + sigma**2)
                    print(f"Given Lognormal parameters: μ = {mu}, σ = {sigma}")
                    print(f"Equivalent Normal Mean: {lognorm_mean}")
                    print(f"Equivalent Normal Variance: {lognorm_variance}")
                else:
                    m = lognormal_param_inputs.children[0].value
                    v = lognormal_param_inputs.children[1].value
                    if m <= 0:
                        raise ValueError("Lognormal mean must be positive.")
                    sigma2 = math.log(1 + v/(m**2))
                    sigma = math.sqrt(sigma2)
                    mu = math.log(m) - sigma2/2
                    print(f"Given Normal parameters: Mean = {m}, Variance = {v}")
                    print(f"Lognormal Mean: {mu}")
                    print(f"Lognormal Std Dev: {sigma}")
        except Exception as e:
            print(f"Error: {e}")

conv_button.on_click(perform_conversion)

# Assemble Conversion Calculator UI
conv_calculator_ui = VBox([
    conv_category,
    conv_inner_box,
    conv_button,
    conv_output
], layout=Layout(margin='10px 0px'))

####################################
# Create Tabs for the Two Calculators
####################################

tab = Tab(children=[dist_calculator_ui, conv_calculator_ui])
tab.set_title(0, "Distributions")
tab.set_title(1, "Conversion")
display(tab)

# **Hypothesis Testing**

In [ ]:
# @title Default title text
import re
import numpy as np
import pandas as pd
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display, clear_output

# For proportions tests, we use statsmodels:
try:
    from statsmodels.stats.proportion import proportions_ztest
except ImportError:
    raise ImportError("Please install statsmodels to run proportions tests (pip install statsmodels)")

# -------------------------------------------------
# Helper Calculation Functions (using built-in libraries)
# -------------------------------------------------

def calc_one_sample_means(data, pop_mean, pop_std):
    n = len(data)
    sample_mean = np.mean(data)
    z = (sample_mean - pop_mean) / (pop_std/np.sqrt(n))
    p_value = 2 * (1 - norm.cdf(abs(z)))
    return z, p_value

def calc_two_sample_means(data1, data2):
    n1, n2 = len(data1), len(data2)
    mean1, mean2 = np.mean(data1), np.mean(data2)
    std1, std2 = np.std(data1, ddof=1), np.std(data2, ddof=1)
    se = np.sqrt(std1**2/n1 + std2**2/n2)
    z = (mean1 - mean2) / se
    p_value = 2 * (1 - norm.cdf(abs(z)))
    return z, p_value

def calc_one_sample_prop(data, pop_prop):
    n = len(data)
    count = np.sum(data)  # assumes binary (0/1)
    z, p_value = proportions_ztest(count, n, value=pop_prop, alternative='two-sided')
    return z, p_value

def calc_two_sample_prop(data1, data2):
    n1, n2 = len(data1), len(data2)
    count1, count2 = np.sum(data1), np.sum(data2)
    z, p_value = proportions_ztest([count1, count2], [n1, n2], alternative='two-sided')
    return z, p_value

# -------------------------------------------------
# UI Builder Functions for Each Test
# -------------------------------------------------

# 1. One Sample z-test (Means)
def build_one_sample_z_means_ui():
    # [Existing code unchanged]
    # (see original version; not modified for file upload since it already has a CSV/Excel option)
    # ... (code unchanged) ...
    # (Code from your original one sample z-test is here.)
    # For brevity, we assume this remains the same as in your file.
    # (Refer to the code above.)
    # Here we just return the container built in your original code.
    # (This function is unchanged.)
    mode_dd = widgets.Dropdown(options=["Data List", "Summary Statistics"],
                               value="Data List", description="Input Mode:")
    tail_dd = widgets.Dropdown(options=["Two Tailed", "One Tailed"],
                               value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    direction_dd = widgets.Dropdown(options=["Greater Than", "Less Than"],
                                    value="Greater Than", description="Direction:", style={'description_width': '150px'})
    direction_box = widgets.HBox([direction_dd], layout=widgets.Layout(visibility='hidden'))
    data_source = widgets.RadioButtons(options=["Upload CSV/Excel", "Manual Entry"],
                                       value="Upload CSV/Excel", description="Data Source:")
    file_upload = widgets.FileUpload(accept='.csv, .xlsx', multiple=False, description="Upload File")
    column_selector = widgets.Dropdown(options=[], description="Select Column:", style={'description_width': '150px'})
    manual_text = widgets.Textarea(value="", placeholder="Enter numbers (comma separated)",
                                   description="Data List:", layout=widgets.Layout(width='50%', height='80px'))
    pop_mean_in = widgets.FloatText(value=0.0, description="Population Mean:", style={'description_width': '150px'})
    pop_std_in = widgets.FloatText(value=1.0, description="Population Std Dev:", style={'description_width': '150px'})
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    data_list_box = widgets.VBox()
    def update_data_source(change):
        if data_source.value == "Upload CSV/Excel":
            data_list_box.children = [file_upload, column_selector,
                                      widgets.HTML("<b>Population Parameters:</b>"),
                                      pop_mean_in, pop_std_in]
        else:
            data_list_box.children = [manual_text,
                                      widgets.HTML("<b>Population Parameters:</b>"),
                                      pop_mean_in, pop_std_in]
    data_source.observe(update_data_source, names='value')
    update_data_source(None)
    sample_mean_in = widgets.FloatText(value=0.0, description="Sample Mean:", style={'description_width': '150px'})
    sample_size_in = widgets.IntText(value=30, description="Sample Size:")
    summary_box = widgets.VBox([widgets.HTML("<b>Enter Summary Statistics:</b>"),
                                pop_mean_in, pop_std_in, sample_mean_in, sample_size_in])
    container = widgets.VBox([mode_dd, tail_dd, direction_box, output_area])
    def update_mode(change):
        if mode_dd.value == "Data List":
            container.children = (mode_dd, tail_dd, direction_box, data_source, data_list_box, sig_level_in, run_button, output_area)
        else:
            container.children = (mode_dd, tail_dd, direction_box, summary_box, sig_level_in, run_button, output_area)
    mode_dd.observe(update_mode, names='value')
    update_mode(None)
    def update_tail_type(change):
        if tail_dd.value == "One Tailed":
            direction_box.layout.visibility = 'visible'
        else:
            direction_box.layout.visibility = 'hidden'
    tail_dd.observe(update_tail_type, names='value')
    update_tail_type(None)
    def update_column_selector(change):
        if file_upload.value:
            uploaded_file = list(file_upload.value.values())[0]
            try:
                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
            except Exception:
                try:
                    df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                except Exception as e:
                    with output_area:
                        clear_output()
                        print("Error reading file:", e)
                    return
            column_selector.options = list(df.columns)
    file_upload.observe(update_column_selector, 'value')
    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                alpha = float(sig_level_in.value)
            except Exception as e:
                print("Invalid significance level:", e)
                return
            if mode_dd.value == "Data List":
                if data_source.value == "Upload CSV/Excel":
                    if file_upload.value:
                        uploaded_file = list(file_upload.value.values())[0]
                        try:
                            try:
                                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
                            except:
                                df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                        except Exception as e:
                            print("Error reading file:", e)
                            return
                        col = column_selector.value
                        try:
                            data = pd.to_numeric(df[col], errors='coerce').dropna().tolist()
                        except Exception as e:
                            print("Error processing data:", e)
                            return
                    else:
                        print("Please upload a file.")
                        return
                    pop_mean = pop_mean_in.value
                    pop_std = pop_std_in.value
                    z, p_val = calc_one_sample_means(data, pop_mean, pop_std)
                    sample_mean_val = np.mean(data)
                else:
                    try:
                        data = [float(x.strip()) for x in manual_text.value.split(",") if x.strip() != ""]
                    except Exception as e:
                        print("Error processing manual data:", e)
                        return
                    pop_mean = pop_mean_in.value
                    pop_std = pop_std_in.value
                    z, p_val = calc_one_sample_means(data, pop_mean, pop_std)
                    sample_mean_val = np.mean(data)
            else:
                pop_mean = pop_mean_in.value
                pop_std = pop_std_in.value
                sample_mean_val = sample_mean_in.value
                n = sample_size_in.value
                z = (sample_mean_val - pop_mean) / (pop_std / np.sqrt(n))
                p_val = 2 * (1 - norm.cdf(abs(z)))
            if tail_dd.value == "One Tailed":
                direction = direction_dd.value
                if direction == "Greater Than":
                    p = 1 - norm.cdf(z)
                    if p < alpha:
                        result_message = (f"The sample mean ({sample_mean_val}) is statistically greater than the population mean ({pop_mean}).")
                    else:
                        result_message = (f"The sample mean ({sample_mean_val}) is NOT statistically greater than the population mean ({pop_mean}).")
                else:
                    p = norm.cdf(z)
                    if p < alpha:
                        result_message = (f"The sample mean ({sample_mean_val}) is statistically less than the population mean ({pop_mean}).")
                    else:
                        result_message = (f"The sample mean ({sample_mean_val}) is NOT statistically less than the population mean ({pop_mean}).")
            else:
                p = p_val
                if p < alpha:
                    result_message = (f"There is a statistically significant difference between the sample mean ({sample_mean_val}) and the population mean ({pop_mean}).")
                else:
                    result_message = (f"There is NO statistically significant difference between the sample mean ({sample_mean_val}) and the population mean ({pop_mean}).")
            print("One Sample z-test (Means):")
            print("z statistic:", z)
            print("p-value:", p)
            print(result_message)
            print(f"At a significance level of {alpha}, we {'reject' if p < alpha else 'fail to reject'} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container

# 2. Two Sample z-test (Means)
def build_two_sample_z_means_ui():
    mode_dd = widgets.Dropdown(options=["Data List", "Summary Statistics"],
                               value="Data List", description="Input Mode:")
    tail_dd = widgets.Dropdown(options=["Two Tailed", "One Tailed"],
                               value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    direction_dd = widgets.Dropdown(options=[("Sample1 > Sample2", "gt"), ("Sample1 < Sample2", "lt")],
                                    value="gt", description="Direction:", style={'description_width': '150px'})
    direction_dd.layout.display = 'none'
    data_source = widgets.RadioButtons(options=["Upload CSV/Excel", "Manual Entry"],
                                        value="Upload CSV/Excel", description="Data Source:")
    file_upload = widgets.FileUpload(accept='.csv, .xlsx', multiple=False, description="Upload File")
    col_sel1 = widgets.Dropdown(options=[], description="Column S1:")
    col_sel2 = widgets.Dropdown(options=[], description="Column S2:")
    manual_text = widgets.Textarea(value="", placeholder="Enter two lists (e.g., [1,2,3], [4,5,6])",
                                   description="Data Lists:", layout=widgets.Layout(width='70%', height='100px'))
    s1_mean = widgets.FloatText(value=0.0, description="Sample 1 Mean:", style={'description_width': '150px'})
    s1_std = widgets.FloatText(value=1.0, description="Sample 1 Std Dev:", style={'description_width': '150px'})
    s1_size = widgets.IntText(value=30, description="Sample 1 Size:", style={'description_width': '150px'})
    s2_mean = widgets.FloatText(value=0.0, description="Sample 2 Mean:", style={'description_width': '150px'})
    s2_std = widgets.FloatText(value=1.0, description="Sample 2 Std Dev:", style={'description_width': '150px'})
    s2_size = widgets.IntText(value=30, description="Sample 2 Size:", style={'description_width': '150px'})
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    data_list_box = widgets.VBox()

    def update_data_source(change):
        if data_source.value == "Upload CSV/Excel":
            data_list_box.children = [file_upload, widgets.HBox([col_sel1, col_sel2])]
        else:
            data_list_box.children = [manual_text]
    data_source.observe(update_data_source, names='value')
    update_data_source(None)

    summary_box = widgets.VBox([s1_mean, s1_std, s1_size, s2_mean, s2_std, s2_size])
    container = widgets.VBox([mode_dd, tail_dd, output_area])

    def update_mode(change):
        if tail_dd.value == "One Tailed":
            direction_dd.layout.display = 'block'
        else:
            direction_dd.layout.display = 'none'
        if mode_dd.value == "Data List":
            container.children = (mode_dd, tail_dd, direction_dd, data_source, data_list_box, sig_level_in, run_button, output_area)
        else:
            container.children = (mode_dd, tail_dd, direction_dd, summary_box, sig_level_in, run_button, output_area)
    mode_dd.observe(update_mode, names='value')
    tail_dd.observe(update_mode, names='value')
    update_mode(None)

    def update_col_selectors(change):
        if file_upload.value:
            uploaded_file = list(file_upload.value.values())[0]
            try:
                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
            except Exception:
                try:
                    df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                except Exception as e:
                    with output_area:
                        clear_output()
                        print("Error reading file:", e)
                    return
            options = list(df.columns)
            col_sel1.options = options
            col_sel2.options = options
    file_upload.observe(update_col_selectors, 'value')

    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                alpha = float(sig_level_in.value)
            except Exception as e:
                print("Invalid significance level:", e)
                return
            if mode_dd.value == "Data List":
                if data_source.value == "Upload CSV/Excel":
                    if file_upload.value:
                        uploaded_file = list(file_upload.value.values())[0]
                        try:
                            try:
                                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
                            except:
                                df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                        except Exception as e:
                            print("Error reading file:", e)
                            return
                        col1 = col_sel1.value
                        col2 = col_sel2.value
                        try:
                            data1 = pd.to_numeric(df[col1], errors='coerce').dropna().tolist()
                            data2 = pd.to_numeric(df[col2], errors='coerce').dropna().tolist()
                        except Exception as e:
                            print("Error processing data:", e)
                            return
                    else:
                        print("Please upload a file.")
                        return
                    z, p_val = calc_two_sample_means(data1, data2)
                else:
                    try:
                        lists_found = re.findall(r'\[([^\]]+)\]', manual_text.value)
                        if len(lists_found) != 2:
                            print("Enter exactly two lists separated by a comma.")
                            return
                        data1 = [float(x.strip()) for x in lists_found[0].split(",") if x.strip() != ""]
                        data2 = [float(x.strip()) for x in lists_found[1].split(",") if x.strip() != ""]
                    except Exception as e:
                        print("Error processing manual data:", e)
                        return
                    z, p_val = calc_two_sample_means(data1, data2)
            else:
                z = (s1_mean.value - s2_mean.value) / np.sqrt(s1_std.value**2/s1_size.value + s2_std.value**2/s2_size.value)
                p_val = 2 * (1 - norm.cdf(abs(z)))

            if tail_dd.value == "One Tailed":
                direction_choice = direction_dd.value
                if direction_choice == "gt":
                    p = 1 - norm.cdf(z)
                    if p < alpha:
                        outcome = "Sample 1 is statistically greater than Sample 2."
                    else:
                        outcome = "No evidence that Sample 1 is statistically greater than Sample 2."
                else:
                    p = norm.cdf(z)
                    if p < alpha:
                        outcome = "Sample 1 is statistically less than Sample 2."
                    else:
                        outcome = "No evidence that Sample 1 is statistically less than Sample 2."
            else:
                p = p_val
                if p < alpha:
                    outcome = "There is a statistically significant difference between the two samples."
                else:
                    outcome = "There is no statistically significant difference between the two samples."

            decision = "reject" if p < alpha else "fail to reject"
            print("Two Sample z-test (Means):")
            print("z statistic:", z)
            print("p-value:", p)
            print(outcome)
            print(f"At a significance level of {alpha}, we {decision} the null hypothesis.")

    run_button.on_click(run_test_action)
    return container

# 3. One Sample z-proportions test (code remains similar)
def build_one_sample_z_proportions_ui():
    pop_prop_in = widgets.FloatText(value=0.5, description="Population Proportion:", style={'description_width': '150px'})
    mode_dd = widgets.Dropdown(options=["Data List", "Summary Statistics"], value="Data List", description="Input Mode:", style={'description_width': '150px'})
    tail_dd = widgets.Dropdown(options=["Two Tailed", "One Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    direction_dd = widgets.Dropdown(options=[("Sample proportion > Population proportion", "gt"),
                                              ("Sample proportion < Population proportion", "lt")],
                                    value="gt", description="Direction:", style={'description_width': '150px'}, layout=widgets.Layout(width='100%'))
    direction_dd.layout.display = 'none'
    data_source = widgets.RadioButtons(options=["Upload CSV/Excel", "Manual Entry"], value="Upload CSV/Excel", description="Data Source:", style={'description_width': '150px'})
    file_upload = widgets.FileUpload(accept='.csv, .xlsx', multiple=False, description="Upload File")
    column_selector = widgets.Dropdown(options=[], description="Select Column:", style={'description_width': '150px'})
    manual_text = widgets.Textarea(value="", placeholder="Enter binary data (0/1) separated by commas", description="Data List:", layout=widgets.Layout(width='50%', height='80px'))
    data_list_box = widgets.VBox([], layout=widgets.Layout(width='100%'))

    def update_data_source(change):
        if data_source.value == "Upload CSV/Excel":
            data_list_box.children = [file_upload, column_selector]
        else:
            data_list_box.children = [manual_text]
    data_source.observe(update_data_source, names='value')
    update_data_source(None)

    sample_prop_in = widgets.FloatText(value=0.0, description="Sample Proportion:", style={'description_width': '150px'})
    sample_size_in = widgets.IntText(value=30, description="Sample Size:", style={'description_width': '150px'})
    summary_box = widgets.VBox([widgets.HTML("<b>Summary Statistics for Proportions:</b>"), pop_prop_in, sample_prop_in, sample_size_in], layout=widgets.Layout(width='100%'))
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output(layout=widgets.Layout(width='100%'))
    run_button = widgets.Button(description="Run Test", button_style='success')

    def update_main_container():
        rows = []
        rows.append(mode_dd)
        rows.append(tail_dd)
        if tail_dd.value == "One Tailed":
            rows.append(direction_dd)
        if mode_dd.value == "Data List":
            rows.append(data_source)
            rows.append(data_list_box)
        else:
            rows.append(summary_box)
        rows.append(sig_level_in)
        rows.append(run_button)
        rows.append(output_area)
        return widgets.VBox(rows)

    container = update_main_container()

    def update_layout(change):
        if tail_dd.value == "One Tailed":
            direction_dd.layout.display = 'block'
        else:
            direction_dd.layout.display = 'none'
        new_container = update_main_container()
        container.children = new_container.children
    mode_dd.observe(update_layout, names='value')
    tail_dd.observe(update_layout, names='value')
    update_layout(None)

    def update_column_selector(change):
        if file_upload.value:
            uploaded_file = list(file_upload.value.values())[0]
            try:
                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
            except Exception:
                try:
                    df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                except Exception as e:
                    with output_area:
                        clear_output()
                        print("Error reading file:", e)
                    return
            column_selector.options = list(df.columns)
    file_upload.observe(update_column_selector, 'value')

    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                alpha = float(sig_level_in.value)
            except Exception as e:
                print("Invalid significance level:", e)
                return
            if mode_dd.value == "Data List":
                if data_source.value == "Upload CSV/Excel":
                    if file_upload.value:
                        uploaded_file = list(file_upload.value.values())[0]
                        try:
                            try:
                                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
                            except:
                                df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                        except Exception as e:
                            print("Error reading file:", e)
                            return
                        col = column_selector.value
                        try:
                            data = pd.to_numeric(df[col], errors='coerce').dropna().tolist()
                        except Exception as e:
                            print("Error processing data:", e)
                            return
                    else:
                        print("Please upload a file.")
                        return
                    pop_prop = pop_prop_in.value
                    z, computed_p = calc_one_sample_prop(data, pop_prop)
                    sample_prop_val = np.mean(data)
                else:
                    try:
                        data = [float(x.strip()) for x in manual_text.value.split(",") if x.strip() != ""]
                    except Exception as e:
                        print("Error processing manual data:", e)
                        return
                    pop_prop = pop_prop_in.value
                    z, computed_p = calc_one_sample_prop(data, pop_prop)
                    sample_prop_val = np.mean(data)
            else:
                pop_prop = pop_prop_in.value
                sample_prop_val = sample_prop_in.value
                n = sample_size_in.value
                se = np.sqrt(pop_prop * (1 - pop_prop) / n)
                z = (sample_prop_val - pop_prop) / se
                computed_p = 2 * (1 - norm.cdf(abs(z)))
            # Corrected one-tailed logic:
            if tail_dd.value == "One Tailed":
                if direction_dd.value == "gt":
                    p = 1 - norm.cdf(z)
                    if p < alpha:
                        outcome = "The sample proportion is statistically greater than the population proportion."
                    else:
                        outcome = "Insufficient evidence for a greater proportion."
                else:
                    p = norm.cdf(z)
                    if p < alpha:
                        outcome = "The sample proportion is statistically less than the population proportion."
                    else:
                        outcome = "Insufficient evidence for a lesser proportion."
            else:
                p = computed_p
                if p < alpha:
                    outcome = "There is a statistically significant difference between the sample and population proportions."
                else:
                    outcome = "No statistically significant difference between the sample and population proportions."
            decision = "reject" if p < alpha else "fail to reject"
            print("One Sample z-proportions test:")
            print("z statistic:", z)
            print("p-value:", p)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container


# 4. Two Sample z-proportions test (code remains similar)
def build_two_sample_z_proportions_ui():
    mode_dd = widgets.Dropdown(options=["Data List", "Summary Statistics"], value="Data List", description="Input Mode:", style={'description_width': '150px'})
    tail_dd = widgets.Dropdown(options=["Two Tailed", "One Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    direction_dd = widgets.Dropdown(options=[("Sample1 > Sample2", "gt"), ("Sample1 < Sample2", "lt")],
                                    value="gt", description="Direction:", style={'description_width': '150px'})
    direction_dd.layout.display = 'none'
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    data_source = widgets.RadioButtons(options=["Upload CSV/Excel", "Manual Entry"], value="Upload CSV/Excel", description="Data Source:", style={'description_width': '150px'})
    file_upload = widgets.FileUpload(accept='.csv, .xlsx', multiple=False, description="Upload File")
    col_sel1 = widgets.Dropdown(options=[], description="Column S1:", style={'description_width': '150px'})
    col_sel2 = widgets.Dropdown(options=[], description="Column S2:", style={'description_width': '150px'})
    manual_text = widgets.Textarea(value="", placeholder="Enter two lists (e.g., [1,0,1], [0,1,1])", description="Data Lists:", layout=widgets.Layout(width='70%', height='100px'))
    data_list_box = widgets.VBox()

    def update_data_source(change):
        if data_source.value == "Upload CSV/Excel":
            data_list_box.children = [file_upload, widgets.HBox([col_sel1, col_sel2])]
        else:
            data_list_box.children = [manual_text]
    data_source.observe(update_data_source, names='value')
    update_data_source(None)

    s1_prop = widgets.FloatText(value=0.0, description="Sample 1 Proportion:", style={'description_width': '150px'})
    s1_size = widgets.IntText(value=30, description="Sample 1 Size:", style={'description_width': '150px'})
    s2_prop = widgets.FloatText(value=0.0, description="Sample 2 Proportion:", style={'description_width': '150px'})
    s2_size = widgets.IntText(value=30, description="Sample 2 Size:", style={'description_width': '150px'})
    summary_box = widgets.VBox([widgets.HTML("<b>Summary Statistics for Proportions:</b>"), s1_prop, s1_size, s2_prop, s2_size])
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')

    def update_main_container():
        rows = []
        rows.append(mode_dd)
        rows.append(tail_dd)
        if tail_dd.value == "One Tailed":
            rows.append(direction_dd)
        if mode_dd.value == "Data List":
            rows.append(data_source)
            rows.append(data_list_box)
        else:
            rows.append(summary_box)
        rows.append(sig_level_in)
        rows.append(run_button)
        rows.append(output_area)
        return widgets.VBox(rows)

    container = update_main_container()

    def update_layout(change):
        if tail_dd.value == "One Tailed":
            direction_dd.layout.display = 'block'
        else:
            direction_dd.layout.display = 'none'
        new_container = update_main_container()
        container.children = new_container.children
    mode_dd.observe(update_layout, names='value')
    tail_dd.observe(update_layout, names='value')
    update_layout(None)

    def update_col_selectors(change):
        if file_upload.value:
            uploaded_file = list(file_upload.value.values())[0]
            try:
                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
            except Exception:
                try:
                    df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                except Exception as e:
                    with output_area:
                        clear_output()
                        print("Error reading file:", e)
                    return
            options = list(df.columns)
            col_sel1.options = options
            col_sel2.options = options
    file_upload.observe(update_col_selectors, 'value')

    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                alpha = float(sig_level_in.value)
            except Exception as e:
                print("Invalid significance level:", e)
                return
            if mode_dd.value == "Data List":
                if data_source.value == "Upload CSV/Excel":
                    if file_upload.value:
                        uploaded_file = list(file_upload.value.values())[0]
                        try:
                            try:
                                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
                            except:
                                df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                        except Exception as e:
                            print("Error reading file:", e)
                            return
                        col1 = col_sel1.value
                        col2 = col_sel2.value
                        try:
                            data1 = pd.to_numeric(df[col1], errors='coerce').dropna().tolist()
                            data2 = pd.to_numeric(df[col2], errors='coerce').dropna().tolist()
                        except Exception as e:
                            print("Error processing data:", e)
                            return
                    else:
                        print("Please upload a file.")
                        return
                    # Compute z and two-tailed p-value for proportions test using proportions_ztest:
                    count1 = np.sum(np.array(data1) == 1)
                    count2 = np.sum(np.array(data2) == 1)
                    n1 = len(data1)
                    n2 = len(data2)
                    z, computed_p = proportions_ztest([count1, count2], [n1, n2], alternative='two-sided')
                else:
                    try:
                        lists_found = re.findall(r'\[([^\]]+)\]', manual_text.value)
                        if len(lists_found) != 2:
                            print("Enter exactly two lists with brackets separated by a comma.")
                            return
                        data1 = [float(x.strip()) for x in lists_found[0].split(",") if x.strip() != ""]
                        data2 = [float(x.strip()) for x in lists_found[1].split(",") if x.strip() != ""]
                    except Exception as e:
                        print("Error processing manual data:", e)
                        return
                    count1 = np.sum(np.array(data1) == 1)
                    count2 = np.sum(np.array(data2) == 1)
                    n1 = len(data1)
                    n2 = len(data2)
                    z, computed_p = proportions_ztest([count1, count2], [n1, n2], alternative='two-sided')
            else:
                count1 = s1_prop.value * s1_size.value
                count2 = s2_prop.value * s2_size.value
                n1 = s1_size.value
                n2 = s2_size.value
                z, computed_p = proportions_ztest([count1, count2], [n1, n2], alternative='two-sided')

            # Corrected one-tailed logic:
            if tail_dd.value == "One Tailed":
                if direction_dd.value == "gt":
                    p = 1 - norm.cdf(z)
                    if p < alpha:
                        outcome = "Sample 1 is statistically greater than Sample 2."
                    else:
                        outcome = "No evidence that Sample 1 is statistically greater than Sample 2."
                else:
                    p = norm.cdf(z)
                    if p < alpha:
                        outcome = "Sample 1 is statistically less than Sample 2."
                    else:
                        outcome = "No evidence that Sample 1 is statistically less than Sample 2."
            else:
                p = computed_p
                if p < alpha:
                    outcome = "There is a statistically significant difference between the two samples."
                else:
                    outcome = "There is no statistically significant difference between the two samples."

            decision = "reject" if p < alpha else "fail to reject"
            print("Two Sample z-proportions test:")
            print("z statistic:", z)
            print("p-value:", p)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")

    run_button.on_click(run_test_action)
    return container

# 5. Fisher's Exact Test
def build_fishers_exact_test_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import fisher_exact
    a_in = widgets.IntText(value=10, description="a:", style={'description_width': '50px'})
    b_in = widgets.IntText(value=5, description="b:", style={'description_width': '50px'})
    c_in = widgets.IntText(value=3, description="c:", style={'description_width': '50px'})
    d_in = widgets.IntText(value=8, description="d:", style={'description_width': '50px'})
    tail_dd = widgets.Dropdown(options=["Two Tailed", "One Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    direction_dd = widgets.Dropdown(options=["Greater", "Less"], value="Greater", description="Direction:", style={'description_width': '150px'})
    direction_box = widgets.HBox([direction_dd])
    def update_tail(change):
        if tail_dd.value == "One Tailed":
            direction_box.layout.visibility = 'visible'
        else:
            direction_box.layout.visibility = 'hidden'
    tail_dd.observe(update_tail, names='value')
    update_tail(None)
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([widgets.HBox([a_in, b_in]), widgets.HBox([c_in, d_in]), tail_dd, direction_box, sig_level_in, run_button, output_area])
    def run_test_action(b):
        with output_area:
            clear_output()
            table = [[a_in.value, b_in.value], [c_in.value, d_in.value]]
            alpha = sig_level_in.value
            if tail_dd.value == "Two Tailed":
                alternative = 'two-sided'
            else:
                alternative = 'greater' if direction_dd.value == "Greater" else 'less'
            oddsratio, p_val = fisher_exact(table, alternative=alternative)
            result_message = "Reject the null hypothesis" if p_val < alpha else "Fail to reject the null hypothesis"
            print("Fisher's Exact Test:")
            print("Odds Ratio:", oddsratio)
            print("p-value:", p_val)
            print(result_message)
    run_button.on_click(run_test_action)
    return container

# 6. One Sample T-test (with Excel upload option)
def build_one_sample_t_test_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import ttest_1samp, t
    import numpy as np
    mode_dd = widgets.Dropdown(options=["Data List", "Summary Statistics"], value="Data List", description="Input Mode:")
    tail_dd = widgets.Dropdown(options=["Two Tailed", "One Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    direction_dd = widgets.Dropdown(options=["Greater", "Less"], value="Greater", description="Direction:", style={'description_width': '150px'})
    direction_box = widgets.HBox([direction_dd])
    def update_tail(change):
        if tail_dd.value == "One Tailed":
            direction_box.layout.visibility = 'visible'
        else:
            direction_box.layout.visibility = 'hidden'
    tail_dd.observe(update_tail, names='value')
    update_tail(None)
    pop_mean_in = widgets.FloatText(value=0.0, description="Hypothesized Mean:", style={'description_width': '150px'})
    # Add Data Source option
    data_source = widgets.RadioButtons(options=["Upload CSV/Excel", "Manual Entry"], value="Upload CSV/Excel", description="Data Source:")
    file_upload = widgets.FileUpload(accept='.csv, .xlsx', multiple=False, description="Upload File")
    column_selector = widgets.Dropdown(options=[], description="Select Column:", style={'description_width': '150px'})
    manual_text = widgets.Textarea(value="", placeholder="Enter numbers separated by commas", description="Data List:", layout=widgets.Layout(width='50%', height='80px'))
    data_list_box = widgets.VBox()
    def update_data_source(change):
        if data_source.value == "Upload CSV/Excel":
            data_list_box.children = [file_upload, column_selector]
        else:
            data_list_box.children = [manual_text]
    data_source.observe(update_data_source, names='value')
    update_data_source(None)
    sample_mean_in = widgets.FloatText(value=0.0, description="Sample Mean:", style={'description_width': '150px'})
    sample_std_in = widgets.FloatText(value=1.0, description="Sample Std Dev:", style={'description_width': '150px'})
    sample_size_in = widgets.IntText(value=30, description="Sample Size:", style={'description_width': '150px'})
    summary_box = widgets.VBox([sample_mean_in, sample_std_in, sample_size_in])
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([mode_dd, tail_dd, direction_box, pop_mean_in, sig_level_in, run_button, output_area])
    def update_mode(change):
        if mode_dd.value == "Data List":
            container.children = [mode_dd, tail_dd, direction_box, pop_mean_in, data_source, data_list_box, sig_level_in, run_button, output_area]
        else:
            container.children = [mode_dd, tail_dd, direction_box, pop_mean_in, summary_box, sig_level_in, run_button, output_area]
    mode_dd.observe(update_mode, names='value')
    update_mode(None)
    def update_column_selector(change):
        if file_upload.value:
            uploaded_file = list(file_upload.value.values())[0]
            try:
                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
            except Exception:
                try:
                    df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                except Exception as e:
                    with output_area:
                        clear_output()
                        print("Error reading file:", e)
                    return
            column_selector.options = list(df.columns)
    file_upload.observe(update_column_selector, 'value')
    def run_test_action(b):
        with output_area:
            clear_output()
            alpha = sig_level_in.value
            mu0 = pop_mean_in.value
            if mode_dd.value == "Data List":
                if data_source.value == "Upload CSV/Excel":
                    if file_upload.value:
                        uploaded_file = list(file_upload.value.values())[0]
                        try:
                            try:
                                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
                            except:
                                df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                        except Exception as e:
                            print("Error reading file:", e)
                            return
                        col = column_selector.value
                        try:
                            data = pd.to_numeric(df[col], errors='coerce').dropna().tolist()
                        except Exception as e:
                            print("Error processing data:", e)
                            return
                    else:
                        print("Please upload a file.")
                        return
                    t_stat, p_val = ttest_1samp(data, mu0)
                    sample_mean_val = np.mean(data)
                    df_val = len(data) - 1
                else:
                    try:
                        data = [float(x.strip()) for x in manual_text.value.split(",") if x.strip() != ""]
                    except Exception as e:
                        print("Error processing manual data:", e)
                        return
                    t_stat, p_val = ttest_1samp(data, mu0)
                    sample_mean_val = np.mean(data)
                    df_val = len(data) - 1
            else:
                mean_val = sample_mean_in.value
                std_val = sample_std_in.value
                n = sample_size_in.value
                t_stat = (mean_val - mu0) / (std_val / np.sqrt(n))
                df_val = n - 1
                p_val = 2 * (1 - t.cdf(abs(t_stat), df=df_val))
                sample_mean_val = mean_val
            if tail_dd.value == "One Tailed":
                if direction_dd.value == "Greater":
                    p = 1 - t.cdf(t_stat, df=df_val)
                    outcome = f"Sample mean ({sample_mean_val}) is significantly greater than {mu0}" if p < alpha else f"No evidence that sample mean is greater than {mu0}"
                else:
                    p = t.cdf(t_stat, df=df_val)
                    outcome = f"Sample mean ({sample_mean_val}) is significantly less than {mu0}" if p < alpha else f"No evidence that sample mean is less than {mu0}"
            else:
                p = p_val
                outcome = f"Sample mean ({sample_mean_val}) is significantly different from {mu0}" if p < alpha else f"No significant difference between sample mean and {mu0}"
            decision = "reject" if p < alpha else "fail to reject"
            print("One Sample T-test:")
            print("t statistic:", t_stat)
            print("p-value:", p)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container

# 7a. Two Sample T-test (with Excel upload option)
def build_two_sample_t_test_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import ttest_ind, t
    import numpy as np
    mode_dd = widgets.Dropdown(options=["Data List", "Summary Statistics"], value="Data List", description="Input Mode:")
    tail_dd = widgets.Dropdown(options=["Two Tailed", "One Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    direction_dd = widgets.Dropdown(options=["Sample1 > Sample2", "Sample1 < Sample2"], value="Sample1 > Sample2", description="Direction:", style={'description_width': '150px'})
    direction_box = widgets.HBox([direction_dd])
    def update_tail(change):
        if tail_dd.value == "One Tailed":
            direction_box.layout.visibility = 'visible'
        else:
            direction_box.layout.visibility = 'hidden'
    tail_dd.observe(update_tail, names='value')
    update_tail(None)
    data_source = widgets.RadioButtons(options=["Upload CSV/Excel", "Manual Entry"], value="Upload CSV/Excel", description="Data Source:")
    file_upload = widgets.FileUpload(accept='.csv, .xlsx', multiple=False, description="Upload File")
    col_sel1 = widgets.Dropdown(options=[], description="Column S1:", style={'description_width': '150px'})
    col_sel2 = widgets.Dropdown(options=[], description="Column S2:", style={'description_width': '150px'})
    manual_text = widgets.Textarea(value="", placeholder="Enter numbers for Sample 1 and Sample 2 separated by brackets", description="Data Lists:", layout=widgets.Layout(width='70%', height='100px'))
    data_list_box = widgets.VBox()
    def update_data_source(change):
        if data_source.value == "Upload CSV/Excel":
            data_list_box.children = [file_upload, widgets.HBox([col_sel1, col_sel2])]
        else:
            data_list_box.children = [manual_text]
    data_source.observe(update_data_source, names='value')
    update_data_source(None)
    s1_mean = widgets.FloatText(value=0.0, description="Sample 1 Mean:", style={'description_width': '150px'})
    s1_std = widgets.FloatText(value=1.0, description="Sample 1 Std Dev:", style={'description_width': '150px'})
    s1_size = widgets.IntText(value=30, description="Sample 1 Size:", style={'description_width': '150px'})
    s2_mean = widgets.FloatText(value=0.0, description="Sample 2 Mean:", style={'description_width': '150px'})
    s2_std = widgets.FloatText(value=1.0, description="Sample 2 Std Dev:", style={'description_width': '150px'})
    s2_size = widgets.IntText(value=30, description="Sample 2 Size:", style={'description_width': '150px'})
    summary_box = widgets.VBox([s1_mean, s1_std, s1_size, s2_mean, s2_std, s2_size])
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([mode_dd, tail_dd, direction_box, sig_level_in, run_button, output_area])
    def update_mode(change):
        if mode_dd.value == "Data List":
            container.children = [mode_dd, tail_dd, direction_box, data_source, data_list_box, sig_level_in, run_button, output_area]
        else:
            container.children = [mode_dd, tail_dd, direction_box, summary_box, sig_level_in, run_button, output_area]
    mode_dd.observe(update_mode, names='value')
    update_mode(None)
    def update_col_selectors(change):
        if file_upload.value:
            uploaded_file = list(file_upload.value.values())[0]
            try:
                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
            except Exception:
                try:
                    df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                except Exception as e:
                    with output_area:
                        clear_output()
                        print("Error reading file:", e)
                    return
            options = list(df.columns)
            col_sel1.options = options
            col_sel2.options = options
    file_upload.observe(update_col_selectors, 'value')
    def run_test_action(b):
        with output_area:
            clear_output()
            alpha = sig_level_in.value
            if mode_dd.value == "Data List":
                if data_source.value == "Upload CSV/Excel":
                    if file_upload.value:
                        uploaded_file = list(file_upload.value.values())[0]
                        try:
                            try:
                                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
                            except:
                                df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                        except Exception as e:
                            print("Error reading file:", e)
                            return
                        col1 = col_sel1.value
                        col2 = col_sel2.value
                        try:
                            data1 = pd.to_numeric(df[col1], errors='coerce').dropna().tolist()
                            data2 = pd.to_numeric(df[col2], errors='coerce').dropna().tolist()
                        except Exception as e:
                            print("Error processing data:", e)
                            return
                    else:
                        print("Please upload a file.")
                        return
                    t_stat, p_val = ttest_ind(data1, data2, equal_var=True)
                    mean1 = np.mean(data1)
                    mean2 = np.mean(data2)
                    df_val = (len(data1) + len(data2) - 2)
                else:
                    try:
                        lists_found = re.findall(r'\[([^\]]+)\]', manual_text.value)
                        if len(lists_found) != 2:
                            print("Enter exactly two lists separated by a comma.")
                            return
                        data1 = [float(x.strip()) for x in lists_found[0].split(",") if x.strip() != ""]
                        data2 = [float(x.strip()) for x in lists_found[1].split(",") if x.strip() != ""]
                    except Exception as e:
                        print("Error processing manual data:", e)
                        return
                    t_stat, p_val = ttest_ind(data1, data2, equal_var=True)
                    mean1 = np.mean(data1)
                    mean2 = np.mean(data2)
                    df_val = (len(data1) + len(data2) - 2)
            else:
                mean1, std1, n1 = s1_mean.value, s1_std.value, s1_size.value
                mean2, std2, n2 = s2_mean.value, s2_std.value, s2_size.value
                t_stat = (mean1 - mean2) / np.sqrt(std1**2/n1 + std2**2/n2)
                df_val = min(n1 - 1, n2 - 1)
                p_val = 2 * (1 - t.cdf(abs(t_stat), df=df_val))
                mean1, mean2 = s1_mean.value, s2_mean.value
            if tail_dd.value == "One Tailed":
                if direction_dd.value == "Sample1 > Sample2":
                    p = 1 - t.cdf(t_stat, df=df_val)
                    outcome = f"Sample 1 mean ({mean1}) is significantly greater than Sample 2 mean ({mean2})" if p < alpha else "No evidence that Sample 1 > Sample 2"
                else:
                    p = t.cdf(t_stat, df=df_val)
                    outcome = f"Sample 1 mean ({mean1}) is significantly less than Sample 2 mean ({mean2})" if p < alpha else "No evidence that Sample 1 < Sample 2"
            else:
                p = p_val
                outcome = "There is a significant difference between the two samples" if p < alpha else "No significant difference between the samples"
            decision = "reject" if p < alpha else "fail to reject"
            print("Two Sample T-test:")
            print("t statistic:", t_stat)
            print("p-value:", p)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container


# 7b. Two Sample T-test (with Excel upload option)
def build_two_sample_welch_t_test_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import ttest_ind, t
    import numpy as np
    mode_dd = widgets.Dropdown(options=["Data List", "Summary Statistics"], value="Data List", description="Input Mode:")
    tail_dd = widgets.Dropdown(options=["Two Tailed", "One Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    direction_dd = widgets.Dropdown(options=["Sample1 > Sample2", "Sample1 < Sample2"], value="Sample1 > Sample2", description="Direction:", style={'description_width': '150px'})
    direction_box = widgets.HBox([direction_dd])
    def update_tail(change):
        if tail_dd.value == "One Tailed":
            direction_box.layout.visibility = 'visible'
        else:
            direction_box.layout.visibility = 'hidden'
    tail_dd.observe(update_tail, names='value')
    update_tail(None)
    data_source = widgets.RadioButtons(options=["Upload CSV/Excel", "Manual Entry"], value="Upload CSV/Excel", description="Data Source:")
    file_upload = widgets.FileUpload(accept='.csv, .xlsx', multiple=False, description="Upload File")
    col_sel1 = widgets.Dropdown(options=[], description="Column S1:", style={'description_width': '150px'})
    col_sel2 = widgets.Dropdown(options=[], description="Column S2:", style={'description_width': '150px'})
    manual_text = widgets.Textarea(value="", placeholder="Enter numbers for Sample 1 and Sample 2 separated by brackets", description="Data Lists:", layout=widgets.Layout(width='70%', height='100px'))
    data_list_box = widgets.VBox()
    def update_data_source(change):
        if data_source.value == "Upload CSV/Excel":
            data_list_box.children = [file_upload, widgets.HBox([col_sel1, col_sel2])]
        else:
            data_list_box.children = [manual_text]
    data_source.observe(update_data_source, names='value')
    update_data_source(None)
    s1_mean = widgets.FloatText(value=0.0, description="Sample 1 Mean:", style={'description_width': '150px'})
    s1_std = widgets.FloatText(value=1.0, description="Sample 1 Std Dev:", style={'description_width': '150px'})
    s1_size = widgets.IntText(value=30, description="Sample 1 Size:", style={'description_width': '150px'})
    s2_mean = widgets.FloatText(value=0.0, description="Sample 2 Mean:", style={'description_width': '150px'})
    s2_std = widgets.FloatText(value=1.0, description="Sample 2 Std Dev:", style={'description_width': '150px'})
    s2_size = widgets.IntText(value=30, description="Sample 2 Size:", style={'description_width': '150px'})
    summary_box = widgets.VBox([s1_mean, s1_std, s1_size, s2_mean, s2_std, s2_size])
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([mode_dd, tail_dd, direction_box, sig_level_in, run_button, output_area])
    def update_mode(change):
        if mode_dd.value == "Data List":
            container.children = [mode_dd, tail_dd, direction_box, data_source, data_list_box, sig_level_in, run_button, output_area]
        else:
            container.children = [mode_dd, tail_dd, direction_box, summary_box, sig_level_in, run_button, output_area]
    mode_dd.observe(update_mode, names='value')
    update_mode(None)
    def update_col_selectors(change):
        if file_upload.value:
            uploaded_file = list(file_upload.value.values())[0]
            try:
                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
            except Exception:
                try:
                    df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                except Exception as e:
                    with output_area:
                        clear_output()
                        print("Error reading file:", e)
                    return
            options = list(df.columns)
            col_sel1.options = options
            col_sel2.options = options
    file_upload.observe(update_col_selectors, 'value')
    def run_test_action(b):
        with output_area:
            clear_output()
            alpha = sig_level_in.value
            if mode_dd.value == "Data List":
                if data_source.value == "Upload CSV/Excel":
                    if file_upload.value:
                        uploaded_file = list(file_upload.value.values())[0]
                        try:
                            try:
                                df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
                            except:
                                df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                        except Exception as e:
                            print("Error reading file:", e)
                            return
                        col1 = col_sel1.value
                        col2 = col_sel2.value
                        try:
                            data1 = pd.to_numeric(df[col1], errors='coerce').dropna().tolist()
                            data2 = pd.to_numeric(df[col2], errors='coerce').dropna().tolist()
                        except Exception as e:
                            print("Error processing data:", e)
                            return
                    else:
                        print("Please upload a file.")
                        return
                    t_stat, p_val = ttest_ind(data1, data2, equal_var=False)
                    mean1 = np.mean(data1)
                    mean2 = np.mean(data2)
                    df_val = (len(data1) + len(data2) - 2)
                else:
                    try:
                        lists_found = re.findall(r'\[([^\]]+)\]', manual_text.value)
                        if len(lists_found) != 2:
                            print("Enter exactly two lists separated by a comma.")
                            return
                        data1 = [float(x.strip()) for x in lists_found[0].split(",") if x.strip() != ""]
                        data2 = [float(x.strip()) for x in lists_found[1].split(",") if x.strip() != ""]
                    except Exception as e:
                        print("Error processing manual data:", e)
                        return
                    t_stat, p_val = ttest_ind(data1, data2, equal_var=False)
                    mean1 = np.mean(data1)
                    mean2 = np.mean(data2)
                    df_val = (len(data1) + len(data2) - 2)
            else:
                mean1, std1, n1 = s1_mean.value, s1_std.value, s1_size.value
                mean2, std2, n2 = s2_mean.value, s2_std.value, s2_size.value
                t_stat = (mean1 - mean2) / np.sqrt(std1**2/n1 + std2**2/n2)
                df_val = min(n1 - 1, n2 - 1)
                p_val = 2 * (1 - t.cdf(abs(t_stat), df=df_val))
                mean1, mean2 = s1_mean.value, s2_mean.value
            if tail_dd.value == "One Tailed":
                if direction_dd.value == "Sample1 > Sample2":
                    p = 1 - t.cdf(t_stat, df=df_val)
                    outcome = f"Sample 1 mean ({mean1}) is significantly greater than Sample 2 mean ({mean2})" if p < alpha else "No evidence that Sample 1 > Sample 2"
                else:
                    p = t.cdf(t_stat, df=df_val)
                    outcome = f"Sample 1 mean ({mean1}) is significantly less than Sample 2 mean ({mean2})" if p < alpha else "No evidence that Sample 1 < Sample 2"
            else:
                p = p_val
                outcome = "There is a significant difference between the two samples" if p < alpha else "No significant difference between the samples"
            decision = "reject" if p < alpha else "fail to reject"
            print("Two Sample T-test:")
            print("t statistic:", t_stat)
            print("p-value:", p)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container

# 8. Paired T-test (remains similar; manual entry only for paired data)
def build_paired_t_test_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import ttest_rel, t
    import numpy as np

    # UI Elements
    mode_dd = widgets.Dropdown(options=["Data List"], value="Data List", description="Input Mode:")
    tail_dd = widgets.Dropdown(options=["Two Tailed", "One Tailed"],
                               value="Two Tailed",
                               description="Tail Type:",
                               style={'description_width': '150px'})
    direction_dd = widgets.Dropdown(options=["Before < After", "Before > After"],
                                    value="Before < After",
                                    description="Direction:",
                                    style={'description_width': '150px'})
    direction_box = widgets.HBox([direction_dd])

    # Show or hide the direction dropdown based on tail selection
    def update_tail(change):
        if tail_dd.value == "One Tailed":
            direction_box.layout.visibility = 'visible'
        else:
            direction_box.layout.visibility = 'hidden'

    tail_dd.observe(update_tail, names='value')
    update_tail(None)

    # Input fields for data and significance level
    data_before = widgets.Textarea(value="",
                                   placeholder="Enter paired values for Before (comma separated)",
                                   description="Before:",layout=widgets.Layout(width='50%', height='100px'))
    data_after = widgets.Textarea(value="",
                                  placeholder="Enter paired values for After (comma separated)",
                                  description="After:",layout=widgets.Layout(width='50%', height='100px'))
    sig_level_in = widgets.FloatText(value=0.05,
                                     description="Significance Level:",
                                     style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([mode_dd, tail_dd, direction_box, data_before, data_after, sig_level_in, run_button, output_area])

    def run_test_action(b):
        with output_area:
            clear_output()
            alpha = sig_level_in.value

            # Parse input data
            try:
                before = [float(x.strip()) for x in data_before.value.split(",") if x.strip() != ""]
                after = [float(x.strip()) for x in data_after.value.split(",") if x.strip() != ""]
            except Exception as e:
                print("Error processing data:", e)
                return

            if len(before) != len(after):
                print("Error: The number of values in 'Before' and 'After' must be the same.")
                return

            # Perform paired t-test (differences computed as before - after)
            t_stat, p_val = ttest_rel(before, after)
            df_val = len(before) - 1

            # Calculate p-value based on selected one-tailed direction or use two-tailed p-value
            if tail_dd.value == "One Tailed":
                if direction_dd.value == "Before < After":
                    # H1: mean(before - after) < 0  -> Lower tail test
                    p = t.cdf(t_stat, df=df_val)
                    outcome = "Before values are significantly less than after values" if p < alpha else "No evidence that Before < After"
                else:
                    # direction "Before > After": H1: mean(before - after) > 0 -> Upper tail test
                    p = 1 - t.cdf(t_stat, df=df_val)
                    outcome = "Before values are significantly greater than after values" if p < alpha else "No evidence that Before > After"
            else:
                p = p_val
                outcome = "There is a significant difference between paired samples" if p < alpha else "No significant difference between paired samples"

            decision = "reject" if p < alpha else "fail to reject"

            # Display the results
            print("Paired T-test:")
            print("t statistic:", t_stat)
            print("p-value:", p)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")

    run_button.on_click(run_test_action)
    return container


# 9. Chi-square Goodness of Fit Test
def build_chi_square_goodness_of_fit_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import chisquare
    obs_text = widgets.Textarea(value="", placeholder="Enter observed frequencies separated by commas", description="Observed:",layout=widgets.Layout(width='50%', height='100px'))
    exp_text = widgets.Textarea(value="", placeholder="Enter expected frequencies separated by commas (optional)", description="Expected:",layout=widgets.Layout(width='50%', height='100px'))
    tail_dd = widgets.Dropdown(options=["Two Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([obs_text, exp_text, tail_dd, sig_level_in, run_button, output_area])
    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                obs = [float(x.strip()) for x in obs_text.value.split(",") if x.strip() != ""]
            except Exception as e:
                print("Error processing observed frequencies:", e)
                return
            try:
                exp = [float(x.strip()) for x in exp_text.value.split(",") if x.strip() != ""]
                if len(exp) != len(obs):
                    print("The number of expected values must match observed values.")
                    return
            except Exception:
                exp = None
            stat, p_val = chisquare(f_obs=obs, f_exp=exp)
            alpha = sig_level_in.value
            outcome = "Significant difference found" if p_val < alpha else "No significant difference"
            decision = "reject" if p_val < alpha else "fail to reject"
            print("Chi-square Goodness of Fit Test:")
            print("Chi-square statistic:", stat)
            print("p-value:", p_val)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container

# 10. Chi-square Test for Independence
def build_chi_square_independence_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import chi2_contingency
    table_text = widgets.Textarea(value="", placeholder="Enter table rows (values separated by commas, one row per line)", description="Table:",layout=widgets.Layout(width='50%', height='150px'))
    tail_dd = widgets.Dropdown(options=["Two Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([table_text, tail_dd, sig_level_in, run_button, output_area])
    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                rows = table_text.value.strip().split("\n")
                table = [[float(x.strip()) for x in row.split(",") if x.strip() != ""] for row in rows if row.strip() != ""]
            except Exception as e:
                print("Error processing table:", e)
                return
            stat, p_val, dof, expected = chi2_contingency(table)
            alpha = sig_level_in.value
            outcome = "Significant association found" if p_val < alpha else "No significant association"
            decision = "reject" if p_val < alpha else "fail to reject"
            print("Chi-square Test for Independence:")
            print("Chi-square statistic:", stat)
            print("p-value:", p_val)
            print("Degrees of Freedom:", dof)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container

# 11. One Way ANOVA
def build_one_way_anova_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import f_oneway
    groups_text = widgets.Textarea(value="", placeholder="Enter group data: for each group, enter numbers separated by commas. Use newlines for different groups.", description="Groups:",layout=widgets.Layout(width='50%', height='150px'))
    tail_dd = widgets.Dropdown(options=["Two Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([groups_text, tail_dd, sig_level_in, run_button, output_area])
    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                groups = []
                rows = groups_text.value.strip().split("\n")
                for row in rows:
                    if row.strip():
                        group = [float(x.strip()) for x in row.split(",") if x.strip() != ""]
                        groups.append(group)
            except Exception as e:
                print("Error processing groups:", e)
                return
            stat, p_val = f_oneway(*groups)
            alpha = sig_level_in.value
            outcome = "Significant difference among groups" if p_val < alpha else "No significant difference among groups"
            decision = "reject" if p_val < alpha else "fail to reject"
            print("One Way ANOVA:")
            print("F statistic:", stat)
            print("p-value:", p_val)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container

# 12. Two Way ANOVA (with both Excel upload and manual entry option)
def build_two_way_anova_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    import pandas as pd
    import statsmodels.api as sm
    from statsmodels.formula.api import ols
    input_mode = widgets.RadioButtons(options=["Upload CSV/Excel", "Manual Entry"], value="Upload CSV/Excel", description="Data Input:")
    # Widgets for file upload
    file_upload = widgets.FileUpload(accept='.csv, .xlsx', multiple=False, description="Upload File")
    dep_var_dd = widgets.Dropdown(options=[], description="Dependent Var:", style={'description_width': '150px'})
    factor_a_dd = widgets.Dropdown(options=[], description="Factor A:", style={'description_width': '150px'})
    factor_b_dd = widgets.Dropdown(options=[], description="Factor B:", style={'description_width': '150px'})
    # Widget for manual entry (a textarea for table data; same format as chi-square table)
    manual_text = widgets.Textarea(value="", placeholder="Enter table rows: values separated by commas, one row per line", description="Manual Data:",layout=widgets.Layout(width='50%', height='150px'))
    tail_dd = widgets.Dropdown(options=["Two Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([input_mode, run_button, output_area])
    def update_inputs(change):
        if input_mode.value == "Upload CSV/Excel":
            container.children = [input_mode, file_upload, dep_var_dd, factor_a_dd, factor_b_dd, tail_dd, sig_level_in, run_button, output_area]
        else:
            container.children = [input_mode, manual_text, tail_dd, sig_level_in, run_button, output_area]
    input_mode.observe(update_inputs, names='value')
    update_inputs(None)
    def update_dropdowns(change):
        if file_upload.value:
            uploaded_file = list(file_upload.value.values())[0]
            try:
                try:
                    df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
                except:
                    df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                options = list(df.columns)
                dep_var_dd.options = options
                factor_a_dd.options = options
                factor_b_dd.options = options
            except Exception as e:
                with output_area:
                    clear_output()
                    print("Error reading file:", e)
    file_upload.observe(update_dropdowns, 'value')
    def run_test_action(b):
        with output_area:
            clear_output()
            alpha = sig_level_in.value
            if input_mode.value == "Upload CSV/Excel":
                if not file_upload.value:
                    print("Please upload a file.")
                    return
                uploaded_file = list(file_upload.value.values())[0]
                try:
                    try:
                        df = pd.read_csv(pd.io.common.BytesIO(uploaded_file['content']))
                    except:
                        df = pd.read_excel(pd.io.common.BytesIO(uploaded_file['content']))
                except Exception as e:
                    print("Error reading file:", e)
                    return
                dep = dep_var_dd.value
                facA = factor_a_dd.value
                facB = factor_b_dd.value
                formula = f'{dep} ~ C({facA}) + C({facB}) + C({facA}):C({facB})'
                try:
                    model = ols(formula, data=df).fit()
                    aov_table = sm.stats.anova_lm(model, typ=2)
                except Exception as e:
                    print("Error performing ANOVA:", e)
                    return
            else:
                try:
                    rows = manual_text.value.strip().split("\n")
                    data = []
                    for row in rows:
                        if row.strip():
                            data.append([float(x.strip()) for x in row.split(",") if x.strip() != ""])
                    # Assume the first column is the dependent variable and the others are factors (for simplicity)
                    # Alternatively, you may parse the input in a different way.
                    df = pd.DataFrame(data[1:], columns=data[0])
                    # For manual entry, we require the user to specify the column names.
                    # For simplicity, we assume columns: 'Y', 'Factor1', 'Factor2'
                    if 'Y' not in df.columns or 'Factor1' not in df.columns or 'Factor2' not in df.columns:
                        print("For manual entry, please include columns: Y, Factor1, Factor2 in the first row.")
                        return
                    formula = 'Y ~ C(Factor1) + C(Factor2) + C(Factor1):C(Factor2)'
                    model = ols(formula, data=df).fit()
                    aov_table = sm.stats.anova_lm(model, typ=2)
                except Exception as e:
                    print("Error processing manual data:", e)
                    return
            outcome = "Significant effects found" if any(aov_table['PR(>F)'] < alpha) else "No significant effects found"
            print("Two Way ANOVA:")
            print(aov_table)
            print(outcome)
    run_button.on_click(run_test_action)
    return container

# 13. Kruskal-Wallis Test
def build_kruskal_wallis_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import kruskal
    groups_text = widgets.Textarea(value="", placeholder="Enter group data: for each group, enter numbers separated by commas; use newlines for different groups.", description="Groups:",layout=widgets.Layout(width='50%', height='150px'))
    tail_dd = widgets.Dropdown(options=["Two Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([groups_text, tail_dd, sig_level_in, run_button, output_area])
    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                groups = []
                rows = groups_text.value.strip().split("\n")
                for row in rows:
                    if row.strip():
                        group = [float(x.strip()) for x in row.split(",") if x.strip() != ""]
                        groups.append(group)
            except Exception as e:
                print("Error processing groups:", e)
                return
            stat, p_val = kruskal(*groups)
            alpha = sig_level_in.value
            outcome = "Significant difference among groups" if p_val < alpha else "No significant difference among groups"
            decision = "reject" if p_val < alpha else "fail to reject"
            print("Kruskal-Wallis Test:")
            print("H statistic:", stat)
            print("p-value:", p_val)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container

# 14. Shapiro-Wilk Test
def build_shapiro_wilk_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import shapiro
    data_text = widgets.Textarea(value="", placeholder="Enter data separated by commas", description="Data:",layout=widgets.Layout(width='50%', height='150px'))
    tail_dd = widgets.Dropdown(options=["Two Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([data_text, tail_dd, sig_level_in, run_button, output_area])
    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                data = [float(x.strip()) for x in data_text.value.split(",") if x.strip() != ""]
            except Exception as e:
                print("Error processing data:", e)
                return
            stat, p_val = shapiro(data)
            alpha = sig_level_in.value
            outcome = "Data is not normally distributed" if p_val < alpha else "Data is normally distributed"
            decision = "reject" if p_val < alpha else "fail to reject"
            print("Shapiro-Wilk Test:")
            print("W statistic:", stat)
            print("p-value:", p_val)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container

# 15. Levene Test
def build_levene_test_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import levene
    groups_text = widgets.Textarea(value="", placeholder="Enter group data: for each group, enter numbers separated by commas; use newlines for different groups.", description="Groups:",layout=widgets.Layout(width='50%', height='150px'))
    tail_dd = widgets.Dropdown(options=["Two Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([groups_text, tail_dd, sig_level_in, run_button, output_area])
    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                groups = []
                rows = groups_text.value.strip().split("\n")
                for row in rows:
                    if row.strip():
                        group = [float(x.strip()) for x in row.split(",") if x.strip() != ""]
                        groups.append(group)
            except Exception as e:
                print("Error processing groups:", e)
                return
            stat, p_val = levene(*groups)
            alpha = sig_level_in.value
            outcome = "Variances are significantly different" if p_val < alpha else "No significant difference in variances"
            decision = "reject" if p_val < alpha else "fail to reject"
            print("Levene Test:")
            print("W statistic:", stat)
            print("p-value:", p_val)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container

# 16. KS Test (Kolmogorov-Smirnov)
def build_ks_test_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import kstest, ks_2samp

    # Dropdown to select test type
    test_type_dd = widgets.Dropdown(
        options=["One-sample", "Two-sample"],
        value="One-sample",
        description="Test Type:",
        style={'description_width': '150px'}
    )

    # Data input widgets
    data_text1 = widgets.Textarea(
        value="",
        placeholder="Enter data separated by commas",
        description="Data 1:",
        layout=widgets.Layout(width='45%', height='150px')
    )
    data_text2 = widgets.Textarea(
        value="",
        placeholder="Enter data separated by commas",
        description="Data 2:",
        layout=widgets.Layout(width='45%', height='150px')
    )

    # Place the two data inputs in an HBox for alignment.
    data_inputs_hbox = widgets.HBox([data_text1, data_text2])

    # Widgets for one-sample test parameters
    mean_in = widgets.FloatText(
        value=0.0,
        description="Mean:",
        style={'description_width': '150px'}
    )
    std_in = widgets.FloatText(
        value=1.0,
        description="Std Dev:",
        style={'description_width': '150px'}
    )

    sig_level_in = widgets.FloatText(
        value=0.05,
        description="Significance Level:",
        style={'description_width': '150px'}
    )

    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')

    # Toggle visibility based on test type
    def toggle_widgets_visibility(change):
        if change['new'] == "Two-sample":
            data_text2.layout.visibility = 'visible'
            mean_in.layout.visibility = 'hidden'
            std_in.layout.visibility = 'hidden'
        else:
            data_text2.layout.visibility = 'hidden'
            mean_in.layout.visibility = 'visible'
            std_in.layout.visibility = 'visible'

    test_type_dd.observe(toggle_widgets_visibility, names='value')
    # Initially hide the second dataset input (space reserved)
    data_text2.layout.visibility = 'hidden'

    container = widgets.VBox([
        test_type_dd,
        data_inputs_hbox,
        mean_in,
        std_in,
        sig_level_in,
        run_button,
        output_area
    ])

    def run_test_action(b):
        with output_area:
            clear_output()
            alpha = sig_level_in.value
            test_type = test_type_dd.value

            if test_type == "One-sample":
                try:
                    data = [float(x.strip()) for x in data_text1.value.split(",") if x.strip() != ""]
                except Exception as e:
                    print("Error processing Data 1:", e)
                    return
                mu = mean_in.value
                sigma = std_in.value
                stat, p_val = kstest(data, 'norm', args=(mu, sigma))
                outcome = ("Data distribution significantly differs from normal"
                           if p_val < alpha else
                           "Data distribution is not significantly different from normal")
                decision = "reject" if p_val < alpha else "fail to reject"
                print("One-sample Kolmogorov-Smirnov Test:")
                print("KS statistic:", stat)
                print("p-value:", p_val)
                print(outcome)
                print(f"At significance level {alpha}, we {decision} the null hypothesis.")

            else:
                try:
                    data1 = [float(x.strip()) for x in data_text1.value.split(",") if x.strip() != ""]
                except Exception as e:
                    print("Error processing Data 1:", e)
                    return
                try:
                    data2 = [float(x.strip()) for x in data_text2.value.split(",") if x.strip() != ""]
                except Exception as e:
                    print("Error processing Data 2:", e)
                    return
                stat, p_val = ks_2samp(data1, data2)
                outcome = ("The two datasets come from significantly different distributions"
                           if p_val < alpha else
                           "The two datasets do not come from significantly different distributions")
                decision = "reject" if p_val < alpha else "fail to reject"
                print("Two-sample Kolmogorov-Smirnov Test:")
                print("KS statistic:", stat)
                print("p-value:", p_val)
                print(outcome)
                print(f"At significance level {alpha}, we {decision} the null hypothesis.")

    run_button.on_click(run_test_action)
    return container

# 17. Pearson Correlation Test
def build_pearson_correlation_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import pearsonr
    import numpy as np
    data1_text = widgets.Textarea(value="", placeholder="Enter data for variable 1 (comma separated)", description="Variable 1:",layout=widgets.Layout(width='50%', height='150px'))
    data2_text = widgets.Textarea(value="", placeholder="Enter data for variable 2 (comma separated)", description="Variable 2:",layout=widgets.Layout(width='50%', height='150px'))
    tail_dd = widgets.Dropdown(options=["Two Tailed", "One Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    direction_dd = widgets.Dropdown(options=["Positive", "Negative"], value="Positive", description="Direction:", style={'description_width': '150px'})
    direction_box = widgets.HBox([direction_dd])
    def update_tail(change):
        if tail_dd.value == "One Tailed":
            direction_box.layout.visibility = 'visible'
        else:
            direction_box.layout.visibility = 'hidden'
    tail_dd.observe(update_tail, names='value')
    update_tail(None)
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([data1_text, data2_text, tail_dd, direction_box, sig_level_in, run_button, output_area])
    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                x = np.array([float(x.strip()) for x in data1_text.value.split(",") if x.strip() != ""])
                y = np.array([float(x.strip()) for x in data2_text.value.split(",") if x.strip() != ""])
            except Exception as e:
                print("Error processing data:", e)
                return
            r, p_val = pearsonr(x, y)
            alpha = sig_level_in.value
            if tail_dd.value == "One Tailed":
                if direction_dd.value == "Positive":
                    p = p_val / 2 if r > 0 else 1 - (p_val / 2)
                    outcome = "Significant positive correlation" if (r > 0 and p < alpha) else "No significant positive correlation"
                else:
                    p = p_val / 2 if r < 0 else 1 - (p_val / 2)
                    outcome = "Significant negative correlation" if (r < 0 and p < alpha) else "No significant negative correlation"
            else:
                p = p_val
                outcome = "Significant correlation" if p < alpha else "No significant correlation"
            decision = "reject" if p < alpha else "fail to reject"
            print("Pearson Correlation Test:")
            print("Correlation coefficient:", r)
            print("p-value:", p)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container

# 18. Spearman Correlation Test
def build_spearman_correlation_ui():
    import ipywidgets as widgets
    from IPython.display import clear_output
    from scipy.stats import spearmanr
    import numpy as np
    data1_text = widgets.Textarea(value="", placeholder="Enter data for variable 1 (comma separated)", description="Variable 1:",layout=widgets.Layout(width='50%', height='150px'))
    data2_text = widgets.Textarea(value="", placeholder="Enter data for variable 2 (comma separated)", description="Variable 2:",layout=widgets.Layout(width='50%', height='150px'))
    tail_dd = widgets.Dropdown(options=["Two Tailed", "One Tailed"], value="Two Tailed", description="Tail Type:", style={'description_width': '150px'})
    direction_dd = widgets.Dropdown(options=["Positive", "Negative"], value="Positive", description="Direction:", style={'description_width': '150px'})
    direction_box = widgets.HBox([direction_dd])
    def update_tail(change):
        if tail_dd.value == "One Tailed":
            direction_box.layout.visibility = 'visible'
        else:
            direction_box.layout.visibility = 'hidden'
    tail_dd.observe(update_tail, names='value')
    update_tail(None)
    sig_level_in = widgets.FloatText(value=0.05, description="Significance Level:", style={'description_width': '150px'})
    output_area = widgets.Output()
    run_button = widgets.Button(description="Run Test", button_style='success')
    container = widgets.VBox([data1_text, data2_text, tail_dd, direction_box, sig_level_in, run_button, output_area])
    def run_test_action(b):
        with output_area:
            clear_output()
            try:
                x = np.array([float(x.strip()) for x in data1_text.value.split(",") if x.strip() != ""])
                y = np.array([float(x.strip()) for x in data2_text.value.split(",") if x.strip() != ""])
            except Exception as e:
                print("Error processing data:", e)
                return
            corr, p_val = spearmanr(x, y)
            alpha = sig_level_in.value
            if tail_dd.value == "One Tailed":
                if direction_dd.value == "Positive":
                    p = p_val / 2 if corr > 0 else 1 - (p_val / 2)
                    outcome = "Significant positive correlation" if (corr > 0 and p < alpha) else "No significant positive correlation"
                else:
                    p = p_val / 2 if corr < 0 else 1 - (p_val / 2)
                    outcome = "Significant negative correlation" if (corr < 0 and p < alpha) else "No significant negative correlation"
            else:
                p = p_val
                outcome = "Significant correlation" if p < alpha else "No significant correlation"
            decision = "reject" if p < alpha else "fail to reject"
            print("Spearman Correlation Test:")
            print("Correlation coefficient:", corr)
            print("p-value:", p)
            print(outcome)
            print(f"At significance level {alpha}, we {decision} the null hypothesis.")
    run_button.on_click(run_test_action)
    return container

# -------------------------------------------------
# Main Container: Parent Dropdown to Select Test
# -------------------------------------------------
main_dropdown = widgets.Dropdown(
    options=[
        "One Sample z-test",
        "Two Sample z-test",
        "One Sample z-proportions test",
        "Two Sample z-proportions test",
        "Fisher's Exact Test",
        "One Sample T-test",
        "Two Sample T-test",
        "Two Sample Welch T-test",
        "Paired T-test",
        "Chi-square Goodness of Fit Test",
        "Chi-square Test for Independence",
        "One Way ANOVA",
        "Two Way ANOVA",
        "Kruskal-Wallis Test",
        "Shapiro-Wilk Test",
        "Levene Test",
        "KS Test",
        "Pearson Correlation",
        "Spearman Correlation"
    ],
    description="Select Test:",
    layout=widgets.Layout(width='23%')
)
test_container = widgets.Output()
def update_test_container(change):
    test_container.clear_output()
    with test_container:
        if main_dropdown.value == "One Sample z-test":
            display(build_one_sample_z_means_ui())
        elif main_dropdown.value == "Two Sample z-test":
            display(build_two_sample_z_means_ui())
        elif main_dropdown.value == "One Sample z-proportions test":
            display(build_one_sample_z_proportions_ui())
        elif main_dropdown.value == "Two Sample z-proportions test":
            display(build_two_sample_z_proportions_ui())
        elif main_dropdown.value == "Fisher's Exact Test":
            display(build_fishers_exact_test_ui())
        elif main_dropdown.value == "One Sample T-test":
            display(build_one_sample_t_test_ui())
        elif main_dropdown.value == "Two Sample T-test":
            display(build_two_sample_t_test_ui())
        elif main_dropdown.value == "Two Sample Welch T-test":
            display(build_two_sample_welch_t_test_ui())
        elif main_dropdown.value == "Paired T-test":
            display(build_paired_t_test_ui())
        elif main_dropdown.value == "Chi-square Goodness of Fit Test":
            display(build_chi_square_goodness_of_fit_ui())
        elif main_dropdown.value == "Chi-square Test for Independence":
            display(build_chi_square_independence_ui())
        elif main_dropdown.value == "One Way ANOVA":
            display(build_one_way_anova_ui())
        elif main_dropdown.value == "Two Way ANOVA":
            display(build_two_way_anova_ui())
        elif main_dropdown.value == "Kruskal-Wallis Test":
            display(build_kruskal_wallis_ui())
        elif main_dropdown.value == "Shapiro-Wilk Test":
            display(build_shapiro_wilk_ui())
        elif main_dropdown.value == "Levene Test":
            display(build_levene_test_ui())
        elif main_dropdown.value == "KS Test":
            display(build_ks_test_ui())
        elif main_dropdown.value == "Pearson Correlation":
            display(build_pearson_correlation_ui())
        elif main_dropdown.value == "Spearman Correlation":
            display(build_spearman_correlation_ui())
main_dropdown.observe(update_test_container, names='value')
update_test_container(None)
display(widgets.VBox([main_dropdown, test_container]))


# **Matrix, Equation solver, Integral, Maxima,Minima**

In [ ]:
# @title Default title text
# Install ipywidgets if needed (uncomment if necessary)
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

import ipywidgets as widgets
from ipywidgets import VBox, HBox, Tab, Layout
import numpy as np
import sympy as sp
import math
from IPython.display import display

# Helper: Check if an expression contains any trigonometric functions
def contains_trig(expr):
    trig_funcs = [sp.sin, sp.cos, sp.tan, sp.csc, sp.sec, sp.cot]
    return any(expr.has(func) for func in trig_funcs)

####################################
# MATRIX OPERATIONS TAB
####################################

matrix_op_dropdown = widgets.Dropdown(
    options=["Addition", "Multiplication", "Inverse", "Determinant"],
    description="Matrix Operation:",
    style={'description_width': '150px'},
    layout=Layout(width='400px')
)

matrix_textarea_A = widgets.Textarea(
    value="1,2,3;4,5,6",
    description="Matrix A:",
    layout=Layout(width='400px', height='100px'),
    style={'description_width': '100px'}
)
matrix_textarea_B = widgets.Textarea(
    value="7,8,9;10,11,12",
    description="Matrix B:",
    layout=Layout(width='400px', height='100px'),
    style={'description_width': '100px'}
)
matrix_textarea = widgets.Textarea(
    value="1,2,3;4,5,6",
    description="Matrix:",
    layout=Layout(width='400px', height='100px'),
    style={'description_width': '100px'}
)

matrix_input_box = VBox()

def update_matrix_inputs(*args):
    op = matrix_op_dropdown.value
    if op in ["Addition", "Multiplication"]:
        matrix_input_box.children = [matrix_textarea_A, matrix_textarea_B]
    else:
        matrix_input_box.children = [matrix_textarea]

update_matrix_inputs()
matrix_op_dropdown.observe(update_matrix_inputs, names="value")

matrix_output = widgets.Output()

def parse_matrix(text):
    try:
        rows = text.strip().split(";")
        matrix = [list(map(float, row.split(","))) for row in rows]
        return np.array(matrix)
    except Exception as e:
        raise ValueError("Invalid matrix format. Use rows separated by ';' and elements by ','.")

def calculate_matrix_operation(b):
    matrix_output.clear_output()
    with matrix_output:
        try:
            op = matrix_op_dropdown.value
            if op in ["Addition", "Multiplication"]:
                A = parse_matrix(matrix_textarea_A.value)
                B = parse_matrix(matrix_textarea_B.value)
                result = A + B if op == "Addition" else np.matmul(A, B)
            else:
                M = parse_matrix(matrix_textarea.value)
                result = np.linalg.inv(M) if op == "Inverse" else np.linalg.det(M)
            print("Result:")
            print(result)
        except Exception as e:
            print("Error:", e)

matrix_calc_button = widgets.Button(
    description="Calculate",
    button_style="success",
    layout=Layout(width='150px')
)
matrix_calc_button.on_click(calculate_matrix_operation)

matrix_tab_ui = VBox([
    matrix_op_dropdown,
    matrix_input_box,
    matrix_calc_button,
    matrix_output
], layout=Layout(margin='10px 0px'))

####################################
# EQUATION SOLVER TAB
####################################

equation_solver_dropdown = widgets.Dropdown(
    options=["2 Variables", "3 Variables", "Quadratic Equation", "Cubic Equation"],
    description="Equation Type:",
    style={'description_width': '150px'},
    layout=Layout(width='400px')
)

# 2 Variables: a1*x + b1*y = c1 and a2*x + b2*y = c2
a1_2 = widgets.FloatText(description="a1:", layout=Layout(width='150px'))
b1_2 = widgets.FloatText(description="b1:", layout=Layout(width='150px'))
c1_2 = widgets.FloatText(description="c1:", layout=Layout(width='150px'))
a2_2 = widgets.FloatText(description="a2:", layout=Layout(width='150px'))
b2_2 = widgets.FloatText(description="b2:", layout=Layout(width='150px'))
c2_2 = widgets.FloatText(description="c2:", layout=Layout(width='150px'))
two_vars_box = VBox([
    HBox([a1_2, b1_2, c1_2]),
    HBox([a2_2, b2_2, c2_2])
])

# 3 Variables: a1*x + b1*y + c1*z = d1, etc.
a1_3 = widgets.FloatText(description="a1:", layout=Layout(width='150px'))
b1_3 = widgets.FloatText(description="b1:", layout=Layout(width='150px'))
c1_3 = widgets.FloatText(description="c1:", layout=Layout(width='150px'))
d1_3 = widgets.FloatText(description="d1:", layout=Layout(width='150px'))
a2_3 = widgets.FloatText(description="a2:", layout=Layout(width='150px'))
b2_3 = widgets.FloatText(description="b2:", layout=Layout(width='150px'))
c2_3 = widgets.FloatText(description="c2:", layout=Layout(width='150px'))
d2_3 = widgets.FloatText(description="d2:", layout=Layout(width='150px'))
a3_3 = widgets.FloatText(description="a3:", layout=Layout(width='150px'))
b3_3 = widgets.FloatText(description="b3:", layout=Layout(width='150px'))
c3_3 = widgets.FloatText(description="c3:", layout=Layout(width='150px'))
d3_3 = widgets.FloatText(description="d3:", layout=Layout(width='150px'))
three_vars_box = VBox([
    HBox([a1_3, b1_3, c1_3, d1_3]),
    HBox([a2_3, b2_3, c2_3, d2_3]),
    HBox([a3_3, b3_3, c3_3, d3_3])
])

# Quadratic Equation: a*x^2 + b*x + c = 0
a_quad = widgets.FloatText(description="a:", layout=Layout(width='150px'))
b_quad = widgets.FloatText(description="b:", layout=Layout(width='150px'))
c_quad = widgets.FloatText(description="c:", layout=Layout(width='150px'))
quad_box = HBox([a_quad, b_quad, c_quad])

# Cubic Equation: a*x^3 + b*x^2 + c*x + d = 0
a_cubic = widgets.FloatText(description="a:", layout=Layout(width='150px'))
b_cubic = widgets.FloatText(description="b:", layout=Layout(width='150px'))
c_cubic = widgets.FloatText(description="c:", layout=Layout(width='150px'))
d_cubic = widgets.FloatText(description="d:", layout=Layout(width='150px'))
cubic_box = HBox([a_cubic, b_cubic, c_cubic, d_cubic])

equation_input_box = VBox()

def update_equation_inputs(*args):
    eq_type = equation_solver_dropdown.value
    if eq_type == "2 Variables":
        equation_input_box.children = [two_vars_box]
    elif eq_type == "3 Variables":
        equation_input_box.children = [three_vars_box]
    elif eq_type == "Quadratic Equation":
        equation_input_box.children = [quad_box]
    else:
        equation_input_box.children = [cubic_box]

update_equation_inputs()
equation_solver_dropdown.observe(update_equation_inputs, names="value")

eq_output = widgets.Output()

def solve_equation(b):
    eq_output.clear_output()
    with eq_output:
        x, y, z = sp.symbols('x y z')
        eq_type = equation_solver_dropdown.value
        try:
            if eq_type == "2 Variables":
                eq1 = sp.Eq(a1_2.value*x + b1_2.value*y, c1_2.value)
                eq2 = sp.Eq(a2_2.value*x + b2_2.value*y, c2_2.value)
                sol = sp.solve([eq1, eq2], (x, y))
            elif eq_type == "3 Variables":
                eq1 = sp.Eq(a1_3.value*x + b1_3.value*y + c1_3.value*z, d1_3.value)
                eq2 = sp.Eq(a2_3.value*x + b2_3.value*y + c2_3.value*z, d2_3.value)
                eq3 = sp.Eq(a3_3.value*x + b3_3.value*y + c3_3.value*z, d3_3.value)
                sol = sp.solve([eq1, eq2, eq3], (x, y, z))
            elif eq_type == "Quadratic Equation":
                sol = sp.solve(a_quad.value*x**2 + b_quad.value*x + c_quad.value, x)
            else:
                sol = sp.solve(a_cubic.value*x**3 + b_cubic.value*x**2 + c_cubic.value*x + d_cubic.value, x)
            print("Solution:", sol)
        except Exception as e:
            print("Error:", e)

eq_solve_button = widgets.Button(
    description="Solve",
    button_style="success",
    layout=Layout(width='150px')
)
eq_solve_button.on_click(solve_equation)

eq_solver_ui = VBox([
    equation_solver_dropdown,
    equation_input_box,
    eq_solve_button,
    eq_output
], layout=Layout(margin='10px 0px'))

####################################
# INTEGRAL CALCULATOR TAB
####################################

integral_function = widgets.Textarea(
    value="x**2",
    description="f(x):",
    layout=Layout(width='400px', height='100px'),
    style={'description_width': '80px'}
)

def append_to_integral(btn):
    integral_function.value += btn.description

integral_buttons = [
    widgets.Button(description="sin(", layout=Layout(width='60px')),
    widgets.Button(description="cos(", layout=Layout(width='60px')),
    widgets.Button(description="tan(", layout=Layout(width='60px')),
    widgets.Button(description="exp(", layout=Layout(width='60px')),
    widgets.Button(description="log(", layout=Layout(width='60px')),
    widgets.Button(description="sqrt(", layout=Layout(width='60px')),
    widgets.Button(description="^", layout=Layout(width='40px'))
]
for btn in integral_buttons:
    btn.on_click(append_to_integral)
integral_buttons_box = HBox(integral_buttons)

integral_lower = widgets.FloatText(
    value=0.0,
    description="Lower Limit:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)
integral_upper = widgets.FloatText(
    value=2.0,
    description="Upper Limit:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)

# For integrals, the "angle unit" applies only if the function contains trigonometric functions.
integral_angle_unit = widgets.RadioButtons(
    options=["Degrees", "Radians"],
    value="Degrees",
    description="Angle Unit:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)

integral_calc_button = widgets.Button(
    description="Calculate Integral",
    button_style="success",
    layout=Layout(width='200px')
)
integral_output = widgets.Output()

def calculate_integral(b):
    integral_output.clear_output()
    with integral_output:
        try:
            x = sp.symbols('x')
            expr = sp.sympify(integral_function.value, convert_xor=True)
            # Only substitute x if the expression contains a trig function and the unit is Degrees.
            if integral_angle_unit.value == "Degrees" and contains_trig(expr):
                expr = expr.subs(x, sp.pi/180 * x)
            lower = integral_lower.value
            upper = integral_upper.value
            result = sp.integrate(expr, (x, lower, upper))
            result_numeric = sp.N(result)  # Evaluate to numeric value
            print("Integral:", result_numeric)
        except Exception as e:
            print("Error:", e)

integral_calc_button.on_click(calculate_integral)

integral_ui = VBox([
    integral_function,
    integral_buttons_box,
    HBox([integral_lower, integral_upper]),
    integral_angle_unit,
    integral_calc_button,
    integral_output
], layout=Layout(margin='10px 0px'))

####################################
# MAXIMA/MINIMA CALCULATOR TAB
####################################

extrema_function = widgets.Textarea(
    value="x**2",
    description="f(x):",
    layout=Layout(width='400px', height='100px'),
    style={'description_width': '80px'}
)

def append_to_extrema(btn):
    extrema_function.value += btn.description

ext_buttons = [
    widgets.Button(description="sin(", layout=Layout(width='60px')),
    widgets.Button(description="cos(", layout=Layout(width='60px')),
    widgets.Button(description="tan(", layout=Layout(width='60px')),
    widgets.Button(description="exp(", layout=Layout(width='60px')),
    widgets.Button(description="log(", layout=Layout(width='60px')),
    widgets.Button(description="sqrt(", layout=Layout(width='60px')),
    widgets.Button(description="^", layout=Layout(width='40px'))
]
for btn in ext_buttons:
    btn.on_click(append_to_extrema)
ext_buttons_box = HBox(ext_buttons)

ext_lower = widgets.FloatText(
    value=-50,
    description="Lower Bound:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)
ext_upper = widgets.FloatText(
    value=50,
    description="Upper Bound:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)

ext_angle_unit = widgets.RadioButtons(
    options=["Degrees", "Radians"],
    value="Degrees",
    description="Angle Unit:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)

ext_calc_button = widgets.Button(
    description="Find Extrema",
    button_style="success",
    layout=Layout(width='150px')
)
ext_output = widgets.Output()

def calculate_extrema(b):
    ext_output.clear_output()
    with ext_output:
        try:
            import numpy as np
            # Parse the function using sympy
            x = sp.symbols('x', real=True)
            f_expr = sp.sympify(extrema_function.value, convert_xor=True)
            lower_bound = float(ext_lower.value)
            upper_bound = float(ext_upper.value)

            # Define a numerical function that respects the selected angle unit.
            if (ext_angle_unit.value == "Degrees" and
                any(f_expr.has(func) for func in [sp.sin, sp.cos, sp.tan, sp.csc, sp.sec, sp.cot])):
                f_lambda = sp.lambdify(x, f_expr, "numpy")
                def f_num(deg):
                    return f_lambda(np.deg2rad(deg))
            else:
                f_num = sp.lambdify(x, f_expr, "numpy")

            # Create a dense grid over the interval.
            xs = np.linspace(lower_bound, upper_bound, 10000)
            ys = f_num(xs)

            # Get the indices from the grid.
            min_idx = np.argmin(ys)
            max_idx = np.argmax(ys)

            # Start candidate set with the grid-found extrema and endpoints.
            candidates = [xs[min_idx], xs[max_idx], lower_bound, upper_bound]

            # If using degrees and a trig function, explicitly add common angles.
            if ext_angle_unit.value == "Degrees" and any(f_expr.has(func) for func in
                                                         [sp.sin, sp.cos, sp.tan, sp.csc, sp.sec, sp.cot]):
                for pt in [-90, 0, 90]:
                    if lower_bound <= pt <= upper_bound:
                        candidates.append(pt)

            # Remove duplicates (using np.unique) and evaluate f at each candidate.
            candidate_vals = {}
            for cand in np.unique(candidates):
                try:
                    val = f_num(cand)
                    candidate_vals[cand] = float(val)
                except Exception as e:
                    print("Error evaluating candidate", cand, ":", e)

            if not candidate_vals:
                print("No valid candidates found.")
                return

            min_candidate = min(candidate_vals.items(), key=lambda t: t[1])
            max_candidate = max(candidate_vals.items(), key=lambda t: t[1])
            print("Minimum at x =", min_candidate[0], "with value =", min_candidate[1])
            print("Maximum at x =", max_candidate[0], "with value =", max_candidate[1])
        except Exception as e:
            print("Error:", e)

ext_calc_button.on_click(calculate_extrema)

extrema_ui = VBox([
    extrema_function,
    ext_buttons_box,
    HBox([ext_lower, ext_upper]),
    ext_angle_unit,
    ext_calc_button,
    ext_output
], layout=Layout(margin='10px 0px'))

####################################
# ASSEMBLE ALL TABS
####################################

all_tabs = Tab(children=[matrix_tab_ui, eq_solver_ui, integral_ui, extrema_ui])
all_tabs.set_title(0, "Matrix Ops")
all_tabs.set_title(1, "Eq Solver")
all_tabs.set_title(2, "Integral")
all_tabs.set_title(3, "Extrema")
display(all_tabs)


# **nCr, nPr, root and power, factorial**

In [ ]:
# @title Default title text
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout
import math, fractions
from IPython.display import display

# Global state
calc_expr = "0"         # the current expression shown on display
pending_operator = None # one of "power", "nth_root", "nCr", "nPr"
stored_value = None     # the first operand (as string)

# Mapping of binary operators to display symbols.
operator_symbols = {
    "power": "^",
    "nth_root": " nrt ",
    "nCr": " C ",
    "nPr": " P "
}

# The calculator display
calc_display = widgets.Text(
    value="0",
    disabled=True,
    layout=Layout(width="400px", height="40px", font_size="20px", text_align="right")
)

def update_display():
    calc_display.value = str(calc_expr)

def clear_all():
    global calc_expr, pending_operator, stored_value
    calc_expr = "0"
    pending_operator = None
    stored_value = None
    update_display()

def append_to_expr(s):
    global calc_expr
    # If display is "0", replace it unless appending an operator.
    if calc_expr == "0":
        if s.strip() in operator_symbols.values():
            calc_expr += s
        else:
            calc_expr = s
    else:
        calc_expr += s
    update_display()

def append_digit(digit):
    append_to_expr(digit)

def append_decimal():
    global calc_expr, pending_operator
    if pending_operator:
        op_sym = operator_symbols[pending_operator]
        parts = calc_expr.split(op_sym)
        current = parts[-1]
    else:
        current = calc_expr
    if "." not in current:
        append_to_expr(".")

def clear_all_button():
    clear_all()

def set_operator(op):
    global calc_expr, pending_operator, stored_value
    if pending_operator is None:
        stored_value = calc_expr
        pending_operator = op
        append_to_expr(operator_symbols[op])

def calculate_result():
    global calc_expr, pending_operator, stored_value
    if pending_operator is None:
        return
    op_sym = operator_symbols[pending_operator]
    try:
        parts = calc_expr.split(op_sym)
        if len(parts) != 2:
            update_display_error("Error")
            return
        first_str = parts[0].strip()
        second_str = parts[1].strip()
        if first_str == "" or second_str == "":
            update_display_error("Error")
            return
        first_num = float(first_str)
        second_num = float(second_str)
        result = None
        if pending_operator == "power":
            result = first_num ** second_num
        elif pending_operator == "nth_root":
            # For nth root, we assume the first number is the index and the second is the radicand.
            # So 4 nth_root 8 should yield 8^(1/4).
            if first_num == 0:
                update_display_error("Error")
                return
            result = second_num ** (1/first_num)
        elif pending_operator == "nCr":
            result = math.comb(int(first_num), int(second_num))
        elif pending_operator == "nPr":
            result = math.perm(int(first_num), int(second_num))
        else:
            update_display_error("Error")
            return
        calc_expr = str(result)
        pending_operator = None
        stored_value = None
        update_display()
    except Exception as e:
        update_display_error("Error")

def update_display_error(msg):
    global calc_expr, pending_operator, stored_value
    calc_expr = msg
    pending_operator = None
    stored_value = None
    update_display()

def unary_operation(func):
    global calc_expr, pending_operator
    try:
        if pending_operator is not None:
            op_sym = operator_symbols[pending_operator]
            parts = calc_expr.split(op_sym)
            if len(parts) == 2 and parts[1].strip() != "":
                second_num = float(parts[1].strip())
                result = func(second_num)
                calc_expr = parts[0] + op_sym + str(result)
            else:
                update_display_error("Error")
                return
        else:
            result = func(float(calc_expr))
            calc_expr = str(result)
        update_display()
    except Exception as e:
        update_display_error("Error")

def inverse_op():
    unary_operation(lambda x: 1/x if x != 0 else float('inf'))

def square_op():
    unary_operation(lambda x: x**2)

def sqrt_op():
    unary_operation(lambda x: math.sqrt(x) if x >= 0 else float('nan'))

def factorial_op():
    unary_operation(lambda x: math.factorial(int(x)) if x >= 0 and int(x) == x else None)

# Enhanced decimal to fraction: show mixed fraction and normal fraction if improper.
def fraction_op():
    global calc_expr
    try:
        num = float(calc_expr)
        frac = fractions.Fraction(num).limit_denominator()
        # Check if improper fraction (absolute numerator greater than denominator)
        if abs(frac.numerator) > frac.denominator:
            whole = abs(frac.numerator) // frac.denominator
            remainder = abs(frac.numerator) % frac.denominator
            sign = "-" if frac.numerator < 0 else ""
            mixed = f"{sign}{whole} {remainder}/{frac.denominator}" if remainder != 0 else f"{sign}{whole}"
            normal = f"{frac.numerator}/{frac.denominator}"
            result_str = f"{mixed} ({normal})"
        else:
            result_str = f"{frac.numerator}/{frac.denominator}"
        calc_expr = result_str
        update_display()
    except Exception as e:
        update_display_error("Error")

def pi_op():
    global calc_expr
    calc_expr = str(math.pi)
    update_display()

def e_op():
    global calc_expr
    calc_expr = str(math.e)
    update_display()

def operator_power(b):
    set_operator("power")

def operator_nth_root(b):
    set_operator("nth_root")

def operator_nCr(b):
    set_operator("nCr")

def operator_nPr(b):
    set_operator("nPr")

def equals_op(b):
    calculate_result()

def clear_button(b):
    clear_all()

# Digit buttons 0-9
digit_buttons = [widgets.Button(description=str(i), layout=Layout(width="60px", height="40px")) for i in range(10)]
for btn in digit_buttons:
    btn.on_click(lambda b, d=btn.description: append_digit(d))

btn_decimal = widgets.Button(description=".", layout=Layout(width="60px", height="40px"))
btn_decimal.on_click(lambda b: append_decimal())

btn_inverse = widgets.Button(description="1/x", layout=Layout(width="60px", height="40px"))
btn_inverse.on_click(lambda b: inverse_op())

btn_square = widgets.Button(description="x²", layout=Layout(width="60px", height="40px"))
btn_square.on_click(lambda b: square_op())

btn_sqrt = widgets.Button(description="√x", layout=Layout(width="60px", height="40px"))
btn_sqrt.on_click(lambda b: sqrt_op())

btn_frac = widgets.Button(description="Frac", layout=Layout(width="60px", height="40px"))
btn_frac.on_click(lambda b: fraction_op())

btn_fact = widgets.Button(description="x!", layout=Layout(width="60px", height="40px"))
btn_fact.on_click(lambda b: factorial_op())

btn_power = widgets.Button(description="^", layout=Layout(width="60px", height="40px"))
btn_power.on_click(operator_power)

btn_nth_root = widgets.Button(description="nrt", layout=Layout(width="60px", height="40px"))
btn_nth_root.on_click(operator_nth_root)

btn_nCr = widgets.Button(description="C", layout=Layout(width="60px", height="40px"))
btn_nCr.on_click(operator_nCr)

btn_nPr = widgets.Button(description="P", layout=Layout(width="60px", height="40px"))
btn_nPr.on_click(operator_nPr)

btn_equals = widgets.Button(description="=", layout=Layout(width="60px", height="40px"), button_style="success")
btn_equals.on_click(equals_op)

btn_pi = widgets.Button(description="π", layout=Layout(width="60px", height="40px"))
btn_pi.on_click(lambda b: pi_op())

btn_e = widgets.Button(description="e", layout=Layout(width="60px", height="40px"))
btn_e.on_click(lambda b: e_op())

btn_clear = widgets.Button(description="C", layout=Layout(width="60px", height="40px"), button_style="danger")
btn_clear.on_click(clear_button)

# Layout the calculator as a grid resembling a regular calculator.
row1 = HBox([calc_display])
row2 = HBox([digit_buttons[7], digit_buttons[8], digit_buttons[9], btn_power, btn_nth_root])
row3 = HBox([digit_buttons[4], digit_buttons[5], digit_buttons[6], btn_nCr, btn_nPr])
row4 = HBox([digit_buttons[1], digit_buttons[2], digit_buttons[3], btn_inverse, btn_square])
row5 = HBox([digit_buttons[0], btn_decimal, btn_frac, btn_fact])
row6 = HBox([btn_pi, btn_e, btn_sqrt, btn_clear, btn_equals])

calculator_ui = VBox([row1, row2, row3, row4, row5, row6])
display(calculator_ui)


# **Trig/Log**

In [ ]:
# @title Default title text
# Install ipywidgets if needed (uncomment if necessary)
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

import ipywidgets as widgets
from ipywidgets import VBox, HBox, Tab, Layout
import math
from IPython.display import display

####################################
# Tab 1: Trigonometric Functions
####################################

# Dropdown for selecting trigonometric function
trig_function_dropdown = widgets.Dropdown(
    options=["sin", "cos", "tan", "cosec", "sec", "cot"],
    description="Function:",
    style={'description_width': '120px'},
    layout=Layout(width='300px')
)

# Radio buttons for angle unit (Radians or Degrees)
angle_unit_radio = widgets.RadioButtons(
    options=["Radians", "Degrees"],
    value="Degrees",
    description="Angle Unit:",
    style={'description_width': '120px'},
    layout=Layout(width='300px')
)

# Input widget for the angle
angle_input = widgets.FloatText(
    description="Angle:",
    value=0.0,
    style={'description_width': '120px'},
    layout=Layout(width='300px')
)

# Button to perform calculation
trig_calc_button = widgets.Button(
    description="Calculate",
    button_style="success",
    layout=Layout(width='150px')
)

# Output widget for trigonometric results
trig_output = widgets.Output()

def calculate_trig(b):
    trig_output.clear_output()
    with trig_output:
        try:
            func = trig_function_dropdown.value
            angle = angle_input.value
            unit = angle_unit_radio.value
            # Convert degrees to radians if needed
            if unit == "Degrees":
                angle = math.radians(angle)
            # Compute the desired trigonometric function
            if func == "sin":
                result = math.sin(angle)
            elif func == "cos":
                result = math.cos(angle)
            elif func == "tan":
                result = math.tan(angle)
            elif func == "cosec":
                s = math.sin(angle)
                if s == 0:
                    raise ValueError("Cosecant undefined when sin(angle)=0")
                result = 1/s
            elif func == "sec":
                c = math.cos(angle)
                if c == 0:
                    raise ValueError("Secant undefined when cos(angle)=0")
                result = 1/c
            elif func == "cot":
                t = math.tan(angle)
                if t == 0:
                    raise ValueError("Cotangent undefined when tan(angle)=0")
                result = 1/t
            print(f"{func}({angle_input.value} {unit}) = {result}")
        except Exception as e:
            print("Error:", e)

trig_calc_button.on_click(calculate_trig)

trig_ui = VBox([
    trig_function_dropdown,
    angle_unit_radio,
    angle_input,
    trig_calc_button,
    trig_output
], layout=Layout(margin='10px 0px'))

####################################
# Tab 2: Logarithms
####################################

# Dropdown for selecting logarithm type
log_type_dropdown = widgets.Dropdown(
    options=[("log base 10", "log10"), ("Natural log (ln)", "ln"), ("Log with custom base", "log_custom")],
    description="Log Type:",
    style={'description_width': '150px'},
    layout=Layout(width='350px')
)

# Input widget for the value to take log of
log_value_input = widgets.FloatText(
    description="Value:",
    value=1.0,
    style={'description_width': '150px'},
    layout=Layout(width='300px')
)

# Input widget for custom base (only visible if custom base is selected)
log_custom_base_input = widgets.FloatText(
    description="Base:",
    value=10.0,
    style={'description_width': '150px'},
    layout=Layout(width='300px')
)

# Function to update visibility of the custom base input
def update_log_inputs(*args):
    if log_type_dropdown.value == "log_custom":
        log_custom_base_input.layout.display = 'block'
    else:
        log_custom_base_input.layout.display = 'none'

log_type_dropdown.observe(update_log_inputs, names="value")
update_log_inputs()

# Button for log calculation
log_calc_button = widgets.Button(
    description="Calculate",
    button_style="success",
    layout=Layout(width='150px')
)

# Output widget for logarithm results
log_output = widgets.Output()

def calculate_log(b):
    log_output.clear_output()
    with log_output:
        try:
            log_type = log_type_dropdown.value
            x = log_value_input.value
            if x <= 0:
                raise ValueError("Logarithm undefined for non-positive values")
            if log_type == "log10":
                result = math.log10(x)
                print(f"log10({x}) = {result}")
            elif log_type == "ln":
                result = math.log(x)
                print(f"ln({x}) = {result}")
            else:
                base = log_custom_base_input.value
                if base <= 0 or base == 1:
                    raise ValueError("Base must be positive and not equal to 1")
                result = math.log(x, base)
                print(f"log base {base} of {x} = {result}")
        except Exception as e:
            print("Error:", e)

log_calc_button.on_click(calculate_log)

log_ui = VBox([
    log_type_dropdown,
    log_value_input,
    log_custom_base_input,
    log_calc_button,
    log_output
], layout=Layout(margin='10px 0px'))

####################################
# Tab 3: Inverse Trigonometric Functions
####################################

# Dropdown for selecting inverse trigonometric function
inv_trig_dropdown = widgets.Dropdown(
    options=[("arcsin", "arcsin"), ("arccos", "arccos"), ("arctan", "arctan"),
             ("arccosec", "arccosec"), ("arcsec", "arcsec"), ("arccot", "arccot")],
    description="Inverse Function:",
    style={'description_width': '150px'},
    layout=Layout(width='350px')
)

# Radio buttons for output unit (Radians or Degrees)
inv_angle_unit_radio = widgets.RadioButtons(
    options=["Radians", "Degrees"],
    value="Degrees",
    description="Result Unit:",
    style={'description_width': '150px'},
    layout=Layout(width='300px')
)

# Input widget for the value for which to compute the inverse function
inv_value_input = widgets.FloatText(
    description="Value:",
    value=0.0,
    style={'description_width': '150px'},
    layout=Layout(width='300px')
)

# Button for inverse trig calculation
inv_trig_calc_button = widgets.Button(
    description="Calculate",
    button_style="success",
    layout=Layout(width='150px')
)

# Output widget for inverse trig results
inv_trig_output = widgets.Output()

def calculate_inv_trig(b):
    inv_trig_output.clear_output()
    with inv_trig_output:
        try:
            func = inv_trig_dropdown.value
            x = inv_value_input.value
            result = None
            if func == "arcsin":
                if x < -1 or x > 1:
                    raise ValueError("arcsin undefined for |x|>1")
                result = math.asin(x)
            elif func == "arccos":
                if x < -1 or x > 1:
                    raise ValueError("arccos undefined for |x|>1")
                result = math.acos(x)
            elif func == "arctan":
                result = math.atan(x)
            elif func == "arccosec":
                if x == 0 or abs(x) < 1:
                    raise ValueError("arccosec undefined for |x|<1 or x=0")
                result = math.asin(1/x)
            elif func == "arcsec":
                if x == 0 or abs(x) < 1:
                    raise ValueError("arcsec undefined for |x|<1 or x=0")
                result = math.acos(1/x)
            elif func == "arccot":
                # Using identity: arccot(x) = π/2 - arctan(x)
                result = math.pi/2 - math.atan(x)
            # Convert result to degrees if requested
            if inv_angle_unit_radio.value == "Degrees":
                result = math.degrees(result)
            print(f"{func}({x}) = {result} {inv_angle_unit_radio.value}")
        except Exception as e:
            print("Error:", e)

inv_trig_calc_button.on_click(calculate_inv_trig)

inv_trig_ui = VBox([
    inv_trig_dropdown,
    inv_angle_unit_radio,
    inv_value_input,
    inv_trig_calc_button,
    inv_trig_output
], layout=Layout(margin='10px 0px'))

####################################
# Assemble Tabs
####################################

tab = Tab(children=[trig_ui, log_ui, inv_trig_ui])
tab.set_title(0, "Trigonometry")
tab.set_title(1, "Logarithms")
tab.set_title(2, "Inverse Trigonometry")
display(tab)
